In [7]:
import numpy as np
import pandas as pd
import dill as dl
import matplotlib.pyplot as plt
import sys
import pickle
import itertools
import re
import os
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as sm_f

In [8]:
pd.set_option("max_columns", None)
pd.set_option("max_rows", None)

# EAS Specification

In [9]:
with open("D:/DZ/Course_6/Diploma/Data/two_time_points.csv", "rb") as inp:
    two_time_points = pickle.load(inp)

In [10]:
two_time_points.head()

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_period
0,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0
1,Владивостокский,Приморский край,1,1,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,1
2,Хасанский район,Приморский край,0,1,2016,NaN,0.0,NaN,4.20,32139.0,NaN,0.023735,0.011735,NaN,0
3,Хасанский район,Приморский край,0,1,2021,NaN,NaN,NaN,6.20,30318.0,NaN,0.020031,0.004033,0.002294,1
4,Артемовский,Приморский край,1,1,2016,NaN,0.0,NaN,2.62,114873.0,NaN,0.009542,0.009249,NaN,0


In [11]:
treat_1_roads = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_roads"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ treatment_period*treatment_1 + C(region)", data=treat_1_roads).fit(cov_type="HC3")
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     11.34
Date:                Tue, 09 May 2023   Prob (F-statistic):           2.82e-14
Time:                        21:30:10   Log-Likelihood:                -933.44
No. Observations:                 209   AIC:                             1887.
Df Residuals:                     199   BIC:                             1920.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            57.9998      4.198     13.815      0.000      49.772      66.228
C(region)[T.Мурманская область]     -18.9624      6.168     -3.074      0.002     -31.051      -6.874
C(region)[T.Новгородская область]    -2.7512      4.973     -0.553      0.580     -12.497       6.995
C(region)[T.Республика Алтай]        22.0785      6.059      3.644      0.000      10.203      33.954
C(region)[T.Республика Марий Эл]     18.7831      4.788      3.923      0.000       9.399      28.167
C(region)[T.Тульская область]        -3.9016      4.879     -0.800      0.424     -13.464       5.660
C(region)[T.Хабаровский край]       -24.4736      6.343     -3.858      0.000     -36.906     -12.041
treatment_period                     -2.0477      3.547     -0.577      0.564      -9.000       4.904
treatment_1                          -3.9092      6.176     -0.633      0.527     -16.014       8.195
treatment_period:treatment_1          0.6595      7.292      0.090      0.928     -13.633      14.952
==============================================================================
Omnibus:                        0.900   Durbin-Watson:                   1.441
Prob(Omnibus):                  0.638   Jarque-Bera (JB):                0.785
Skew:                          -0.150   Prob(JB):                        0.675
Kurtosis:                       3.011   Cond. No.                         7.95
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [12]:
treat_1_roads = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_roads"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ treatment_period*treatment_1", data=treat_1_roads).fit(cov_type="cluster", cov_kwds={'groups': treat_1_roads['region']})
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                     3.195
Date:                Tue, 09 May 2023   Prob (F-statistic):              0.105
Time:                        21:30:39   Log-Likelihood:                -974.38
No. Observations:                 209   AIC:                             1957.
Df Residuals:                     205   BIC:                             1970.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       58.0953      6.497      8.942      0.000      45.361      70.829
treatment_period                -3.6253      1.548     -2.342      0.019      -6.659      -0.592
treatment_1                     -2.8301      7.747     -0.365      0.715     -18.014      12.354
treatment_period:treatment_1    -1.2145      3.457     -0.351      0.725      -7.989       5.560
==============================================================================
Omnibus:                       22.190   Durbin-Watson:                   0.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                8.878
Skew:                          -0.257   Prob(JB):                       0.0118
Kurtosis:                       2.131   Cond. No.                         6.70
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [13]:
treat_1_schools = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_schools"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ treatment_period*treatment_1", data=treat_1_schools).fit(cov_type="cluster", cov_kwds={'groups': treat_1_schools['region']})
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                    0.5768
Date:                Tue, 09 May 2023   Prob (F-statistic):              0.648
Time:                        21:30:48   Log-Likelihood:                -636.72
No. Observations:                 158   AIC:                             1281.
Df Residuals:                     154   BIC:                             1294.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        3.8329      1.640      2.337      0.019       0.619       7.047
treatment_period                 6.3443      6.187      1.025      0.305      -5.783      18.471
treatment_1                     -1.5722      1.963     -0.801      0.423      -5.421       2.276
treatment_period:treatment_1     0.6814      3.853      0.177      0.860      -6.870       8.233
==============================================================================
Omnibus:                      133.450   Durbin-Watson:                   2.083
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1389.067
Skew:                           3.153   Prob(JB):                    2.33e-302
Kurtosis:                      16.085   Cond. No.                         5.92
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [14]:
treat_1_schools = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_schools"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ treatment_period*treatment_1 + C(region)", data=treat_1_schools).fit(cov_type="HC3")
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     1.157
Date:                Tue, 09 May 2023   Prob (F-statistic):              0.325
Time:                        21:30:50   Log-Likelihood:                -615.95
No. Observations:                 158   AIC:                             1254.
Df Residuals:                     147   BIC:                             1288.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            33.1483     19.565      1.694      0.090      -5.198      71.494
C(region)[T.Мурманская область]     -35.8414     74.551     -0.481      0.631    -181.960     110.277
C(region)[T.Новгородская область]   -29.5358     19.752     -1.495      0.135     -68.249       9.178
C(region)[T.Приморский край]        -29.1740     19.881     -1.467      0.142     -68.140       9.792
C(region)[T.Республика Алтай]       -29.6196     20.405     -1.452      0.147     -69.612      10.373
C(region)[T.Республика Марий Эл]    -35.6300     20.177     -1.766      0.077     -75.176       3.916
C(region)[T.Тульская область]       -33.0224     19.952     -1.655      0.098     -72.128       6.083
C(region)[T.Хабаровский край]       -11.4817     26.174     -0.439      0.661     -62.782      39.819
treatment_period                      8.6721      3.869      2.242      0.025       1.089      16.255
treatment_1                           0.5163      1.853      0.279      0.780      -3.115       4.147
treatment_period:treatment_1         -3.5953      5.988     -0.600      0.548     -15.332       8.141
==============================================================================
Omnibus:                       77.844   Durbin-Watson:                   2.495
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              296.105
Skew:                           1.908   Prob(JB):                     5.03e-65
Kurtosis:                       8.515   Cond. No.                         21.2
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [15]:
treat_1_sewerage = two_time_points[["raion", "region", "treatment_1", "treatment_period", "sewage_grid_fixed"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ treatment_period*treatment_1", data=treat_1_sewerage).fit(cov_type="cluster", cov_kwds={'groups': treat_1_sewerage['region']})
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     3.943
Date:                Tue, 09 May 2023   Prob (F-statistic):             0.0720
Time:                        21:31:14   Log-Likelihood:                 159.86
No. Observations:                 103   AIC:                            -311.7
Df Residuals:                      99   BIC:                            -301.2
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0217      0.007      3.083      0.002       0.008       0.035
treatment_period                 0.0098      0.018      0.546      0.585      -0.025       0.045
treatment_1                     -0.0049      0.006     -0.772      0.440      -0.017       0.008
treatment_period:treatment_1    -0.0188      0.017     -1.085      0.278      -0.053       0.015
==============================================================================
Omnibus:                      145.171   Durbin-Watson:                   2.183
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4423.498
Skew:                           5.147   Prob(JB):                         0.00
Kurtosis:                      33.410   Cond. No.                         6.81
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [24]:
treat_1_sewerage = two_time_points[["raion", "region", "treatment_1", "treatment_period", "sewage_grid_fixed"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ treatment_period*treatment_1 + C(region)", data=treat_1_sewerage).fit(cov_type="HC3")
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                 -0.031
Method:                 Least Squares   F-statistic:                    0.7598
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.653
Time:                        18:46:21   Log-Likelihood:                 161.50
No. Observations:                 103   AIC:                            -303.0
Df Residuals:                      93   BIC:                            -276.7
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0176      0.012      1.448      0.148      -0.006       0.041
C(region)[T.Мурманская область]       0.0198      0.029      0.677      0.498      -0.038       0.077
C(region)[T.Новгородская область]    -0.0046      0.008     -0.584      0.559      -0.020       0.011
C(region)[T.Приморский край]          0.0141      0.014      1.003      0.316      -0.013       0.042
C(region)[T.Республика Алтай]         0.0076      0.098      0.077      0.938      -0.185       0.200
C(region)[T.Тульская область]        -0.0042      0.010     -0.442      0.659      -0.023       0.015
C(region)[T.Хабаровский край]         0.0008      0.009      0.086      0.931      -0.017       0.019
treatment_period                      0.0105      0.017      0.623      0.533      -0.023       0.044
treatment_1                          -0.0079      0.016     -0.478      0.633      -0.040       0.024
treatment_period:treatment_1         -0.0187      0.021     -0.913      0.361      -0.059       0.021
==============================================================================
Omnibus:                      136.566   Durbin-Watson:                   2.197
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3485.252
Skew:                           4.726   Prob(JB):                         0.00
Kurtosis:                      29.884   Cond. No.                         16.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [16]:
treat_1_heat = two_time_points[["raion", "region", "treatment_1", "treatment_period", "heat_grid_fixed"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ treatment_period*treatment_1", data=treat_1_heat).fit(cov_type="HC3") # т.к. в выборке только один регион
heat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                 -0.035
Method:                 Least Squares   F-statistic:                    0.3414
Date:                Tue, 09 May 2023   Prob (F-statistic):              0.795
Time:                        21:31:24   Log-Likelihood:                 126.39
No. Observations:                  53   AIC:                            -244.8
Df Residuals:                      49   BIC:                            -236.9
Df Model:                           3                                         
Covariance Type:                  HC3                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0240      0.004      5.779      0.000       0.016       0.032
treatment_period                 0.0087      0.011      0.787      0.431      -0.013       0.030
treatment_1                      0.0015      0.007      0.213      0.831      -0.012       0.015
treatment_period:treatment_1    -0.0048      0.014     -0.355      0.723      -0.031       0.022
==============================================================================
Omnibus:                       36.678   Durbin-Watson:                   1.732
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               86.986
Skew:                           2.099   Prob(JB):                     1.29e-19
Kurtosis:                       7.666   Cond. No.                         6.15
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [26]:
treat_1_heat = two_time_points[["raion", "region", "treatment_1", "treatment_period", "heat_grid_fixed"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ treatment_period*treatment_1 + C(region)", data=treat_1_heat).fit(cov_type="HC3") # т.к. в выборке только один регион
heat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                 -0.035
Method:                 Least Squares   F-statistic:                    0.3414
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.795
Time:                        18:46:25   Log-Likelihood:                 126.39
No. Observations:                  53   AIC:                            -244.8
Df Residuals:                      49   BIC:                            -236.9
Df Model:                           3                                         
Covariance Type:                  HC3                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0240      0.004      5.779      0.000       0.016       0.032
treatment_period                 0.0087      0.011      0.787      0.431      -0.013       0.030
treatment_1                      0.0015      0.007      0.213      0.831      -0.012       0.015
treatment_period:treatment_1    -0.0048      0.014     -0.355      0.723      -0.031       0.022
==============================================================================
Omnibus:                       36.678   Durbin-Watson:                   1.732
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               86.986
Skew:                           2.099   Prob(JB):                     1.29e-19
Kurtosis:                       7.666   Cond. No.                         6.15
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [17]:
treat_1_water = two_time_points[["raion", "region", "treatment_1", "treatment_period", "water_grid_fixed"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ treatment_period*treatment_1", data=treat_1_water).fit(cov_type="cluster", cov_kwds={'groups': treat_1_water['region']})
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.064
Date:                Tue, 09 May 2023   Prob (F-statistic):              0.224
Time:                        21:31:39   Log-Likelihood:                 246.95
No. Observations:                 167   AIC:                            -485.9
Df Residuals:                     163   BIC:                            -473.4
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0319      0.015      2.179      0.029       0.003       0.061
treatment_period                -0.0130      0.014     -0.961      0.337      -0.040       0.014
treatment_1                     -0.0196      0.015     -1.317      0.188      -0.049       0.010
treatment_period:treatment_1     0.0170      0.014      1.230      0.219      -0.010       0.044
==============================================================================
Omnibus:                      322.807   Durbin-Watson:                   1.885
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            93630.930
Skew:                          10.005   Prob(JB):                         0.00
Kurtosis:                     117.261   Cond. No.                         6.27
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [28]:
treat_1_water = two_time_points[["raion", "region", "treatment_1", "treatment_period", "water_grid_fixed"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ treatment_period*treatment_1 + C(region)", data=treat_1_water).fit(cov_type="HC3")
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     3.189
Date:                Tue, 25 Apr 2023   Prob (F-statistic):            0.00221
Time:                        18:46:30   Log-Likelihood:                 252.91
No. Observations:                 167   AIC:                            -487.8
Df Residuals:                     158   BIC:                            -459.8
Df Model:                           8                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0236      0.008      2.843      0.004       0.007       0.040
C(region)[T.Мурманская область]       0.0412      0.034      1.226      0.220      -0.025       0.107
C(region)[T.Новгородская область]    -0.0065      0.005     -1.376      0.169      -0.016       0.003
C(region)[T.Приморский край]          0.0052      0.005      1.076      0.282      -0.004       0.015
C(region)[T.Республика Алтай]         0.0058      0.007      0.839      0.401      -0.008       0.019
C(region)[T.Хабаровский край]         0.0178      0.007      2.531      0.011       0.004       0.031
treatment_period                     -0.0123      0.012     -1.056      0.291      -0.035       0.010
treatment_1                          -0.0211      0.014     -1.547      0.122      -0.048       0.006
treatment_period:treatment_1          0.0161      0.013      1.243      0.214      -0.009       0.041
==============================================================================
Omnibus:                      313.990   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            82059.103
Skew:                           9.487   Prob(JB):                         0.00
Kurtosis:                     109.925   Cond. No.                         8.02
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [18]:
with open("D:/DZ/Course_6/Diploma/Data/financial_data.csv", "rb") as inp:
    temp_fin = pickle.load(inp)

In [19]:
with open("D:/DZ/Course_5/Курсовая/data/data_for_models.csv", "rb") as inp:
    data_for_models = pickle.load(inp)

In [20]:
data_for_models.head()

,raion,region,treatment_1,treatment_2
0,Владивостокский,Приморский край,1,1
1,Хасанский район,Приморский край,0,1
2,Артемовский,Приморский край,1,1
3,Шкотовский район,Приморский край,1,0
4,Надеждинский район,Приморский край,0,1


In [21]:
mapping = {"Приморский край": 2016, "Мурманская область": 2016, "Костромская область": 2015, "Республика Алтай": 2014, "Республика Марий Эл": 2014, "Хабаровский край": 2014, 
           "Тульская область": 2014, "Новгородская область": 2016}

In [22]:
actual, initial, difference = temp_fin.loc[temp_fin.budget_type == "actual_spending"], temp_fin.loc[temp_fin.budget_type == "initial_budget"], temp_fin.loc[temp_fin.budget_type == "разница"]

In [23]:
actual = pd.pivot(actual, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
initial = pd.pivot(initial, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
difference = pd.pivot(difference, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()

In [24]:
for i in actual, initial, difference:
    i.rename(columns={"дорожное хозяйство (дорожные фонды)": "roads", "жилищно-коммунальное хозяйство": "housing", "образование": "education"}, inplace=True)

In [25]:
actual.head()

sub_indicator,region,raion,year,roads,housing,education
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN


In [26]:
frames = [actual, initial, difference]

In [27]:
for i in range(len(frames)):
    temp_fin_data = pd.DataFrame()
    for j in mapping.keys():
        temp_fin_data = temp_fin_data.append(frames[i].loc[(frames[i].region == j) & ((frames[i].year == mapping[j]) | (frames[i].year == mapping[j] + 5))])
    temp_fin_data["treatment_period"] = [1 if i > 2018 else 0 for i in temp_fin_data.year]
    frames[i] = pd.merge(temp_fin_data, data_for_models, on=["region", "raion"], how="inner")

In [28]:
#frames[0]

In [29]:
actual, initial, difference = frames

In [30]:
actual.head()

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2
0,Приморский край,Анучинский район,2016,1915.738966,233.152838,11025.123600,0,0,1
1,Приморский край,Анучинский район,2021,NaN,NaN,NaN,1,0,1
2,Приморский край,Арсеньевский,2016,928.610553,1899.621063,9445.185699,0,0,0
3,Приморский край,Арсеньевский,2021,726.384117,3508.931747,9828.533690,1,0,0
4,Приморский край,Артемовский,2016,59.130107,1628.357748,8157.681600,0,1,1


### Models for actual budgets

In [31]:
actual_roads_data = actual[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=actual_roads_data).fit(cov_type="cluster", cov_kwds={'groups': actual_roads_data['region']})
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     8.869
Date:                Tue, 09 May 2023   Prob (F-statistic):            0.00879
Time:                        21:32:23   Log-Likelihood:                -2360.8
No. Observations:                 271   AIC:                             4730.
Df Residuals:                     267   BIC:                             4744.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1002.3017    203.638      4.922      0.000     603.179    1401.424
treatment_period               488.4615    265.295      1.841      0.066     -31.508    1008.431
treatment_1                    208.8738    234.194      0.892      0.372    -250.138     667.886
treatment_period:treatment_1   134.4433    206.072      0.652      0.514    -269.449     538.336
==============================================================================
Omnibus:                      189.914   Durbin-Watson:                   1.591
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2028.101
Skew:                           2.761   Prob(JB):                         0.00
Kurtosis:                      15.211   Cond. No.                         6.29
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [43]:
actual_roads_data = actual[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=actual_roads_data).fit(cov_type="HC3")
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     4.838
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           2.14e-06
Time:                        18:47:03   Log-Likelihood:                -2345.7
No. Observations:                 271   AIC:                             4713.
Df Residuals:                     260   BIC:                             4753.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1383.0373    327.580      4.222      0.000     740.993    2025.082
C(region)[T.Мурманская область]     469.6249    545.372      0.861      0.389    -599.285    1538.534
C(region)[T.Новгородская область]  -980.4289    368.902     -2.658      0.008   -1703.463    -257.395
C(region)[T.Приморский край]       -727.9253    341.665     -2.131      0.033   -1397.576     -58.275
C(region)[T.Республика Алтай]       529.1515    505.298      1.047      0.295    -461.214    1519.517
C(region)[T.Республика Марий Эл]   -682.2248    387.627     -1.760      0.078   -1441.960      77.511
C(region)[T.Тульская область]      -266.6133    430.033     -0.620      0.535   -1109.463     576.236
C(region)[T.Хабаровский край]      -198.6549    438.180     -0.453      0.650   -1057.472     660.163
treatment_period                    435.5981    225.449      1.932      0.053      -6.274     877.471
treatment_1                         172.6209    253.564      0.681      0.496    -324.355     669.597
treatment_period:treatment_1        183.8305    382.311      0.481      0.631    -565.486     933.147
==============================================================================
Omnibus:                      191.099   Durbin-Watson:                   1.765
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2285.725
Skew:                           2.731   Prob(JB):                         0.00
Kurtosis:                      16.138   Cond. No.                         9.51
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [44]:
actual_housing_data = actual[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=actual_housing_data).fit(cov_type="cluster", cov_kwds={'groups': actual_housing_data['region']})
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1.467
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.304
Time:                        18:47:05   Log-Likelihood:                -2912.1
No. Observations:                 270   AIC:                             5832.
Df Residuals:                     266   BIC:                             5847.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3624.2584   2250.999      1.610      0.107    -787.618    8036.135
treatment_period              2824.2381   2011.402      1.404      0.160   -1118.037    6766.513
treatment_1                  -1706.8698   2105.715     -0.811      0.418   -5833.996    2420.256
treatment_period:treatment_1 -2505.7811   2196.313     -1.141      0.254   -6810.475    1798.912
==============================================================================
Omnibus:                      378.619   Durbin-Watson:                   1.137
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33098.412
Skew:                           6.747   Prob(JB):                         0.00
Kurtosis:                      55.535   Cond. No.                         6.31
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [45]:
actual_housing_data = actual[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=actual_housing_data).fit(cov_type="HC3")
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     5.016
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.14e-06
Time:                        18:47:07   Log-Likelihood:                -2874.6
No. Observations:                 270   AIC:                             5771.
Df Residuals:                     259   BIC:                             5811.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1137.2212    868.531      1.309      0.190    -565.069    2839.511
C(region)[T.Мурманская область]    1995.0222    892.535      2.235      0.025     245.687    3744.358
C(region)[T.Новгородская область] -1397.7959    508.481     -2.749      0.006   -2394.401    -401.191
C(region)[T.Приморский край]       -216.7397    472.180     -0.459      0.646   -1142.195     708.716
C(region)[T.Республика Алтай]       675.3914    690.536      0.978      0.328    -678.034    2028.817
C(region)[T.Республика Марий Эл]   -617.8045    490.007     -1.261      0.207   -1578.201     342.592
C(region)[T.Тульская область]      1064.2575    698.426      1.524      0.128    -304.633    2433.148
C(region)[T.Хабаровский край]      1.646e+04   4397.195      3.744      0.000    7842.589    2.51e+04
treatment_period                   2325.7773   1751.183      1.328      0.184   -1106.479    5758.033
treatment_1                        -270.5223    962.270     -0.281      0.779   -2156.536    1615.491
treatment_period:treatment_1      -2091.9235   2012.032     -1.040      0.298   -6035.434    1851.587
==============================================================================
Omnibus:                      358.636   Durbin-Watson:                   1.506
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30663.463
Skew:                           6.091   Prob(JB):                         0.00
Kurtosis:                      53.767   Cond. No.                         9.83
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [46]:
actual_education_data = actual[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=actual_education_data).fit(cov_type="cluster", cov_kwds={'groups': actual_education_data['region']})
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                    0.8874
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.493
Time:                        18:47:09   Log-Likelihood:                -2878.6
No. Observations:                 273   AIC:                             5765.
Df Residuals:                     269   BIC:                             5780.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1.567e+04   2935.073      5.338      0.000    9913.427    2.14e+04
treatment_period              1339.0344   1726.164      0.776      0.438   -2044.184    4722.253
treatment_1                  -3653.9746   2601.194     -1.405      0.160   -8752.222    1444.272
treatment_period:treatment_1    32.7812    724.176      0.045      0.964   -1386.578    1452.140
==============================================================================
Omnibus:                      165.380   Durbin-Watson:                   0.765
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1172.722
Skew:                           2.441   Prob(JB):                    2.22e-255
Kurtosis:                      11.903   Cond. No.                         6.31
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [47]:
actual_education_data = actual[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=actual_education_data).fit(cov_type="HC3")
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     35.52
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           2.67e-43
Time:                        18:47:11   Log-Likelihood:                -2796.8
No. Observations:                 273   AIC:                             5616.
Df Residuals:                     262   BIC:                             5655.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.537e+04   1073.867     14.316      0.000    1.33e+04    1.75e+04
C(region)[T.Мурманская область]    8536.4969   1421.758      6.004      0.000    5749.903    1.13e+04
C(region)[T.Новгородская область] -6088.5038    972.744     -6.259      0.000   -7995.047   -4181.960
C(region)[T.Приморский край]      -2635.7170   1139.557     -2.313      0.021   -4869.207    -402.227
C(region)[T.Республика Алтай]      7588.1624   1798.233      4.220      0.000    4063.691    1.11e+04
C(region)[T.Республика Марий Эл]  -5713.7485   1002.949     -5.697      0.000   -7679.492   -3748.005
C(region)[T.Тульская область]     -2760.6913   1136.834     -2.428      0.015   -4988.844    -532.538
C(region)[T.Хабаровский край]      1.124e+04   2661.873      4.222      0.000    6021.920    1.65e+04
treatment_period                    487.1777   1069.556      0.455      0.649   -1609.114    2583.469
treatment_1                       -2724.9325   1095.786     -2.487      0.013   -4872.634    -577.231
treatment_period:treatment_1        665.1845   1743.639      0.381      0.703   -2752.285    4082.654
==============================================================================
Omnibus:                      170.281   Durbin-Watson:                   1.348
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1763.888
Skew:                           2.358   Prob(JB):                         0.00
Kurtosis:                      14.525   Cond. No.                         9.54
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

### Models for initial budgets

In [48]:
initial_roads_data = initial[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=initial_roads_data).fit(cov_type="cluster", cov_kwds={'groups': initial_roads_data['region']})
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     24.01
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           0.000966
Time:                        18:47:13   Log-Likelihood:                -1834.9
No. Observations:                 215   AIC:                             3678.
Df Residuals:                     211   BIC:                             3691.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      367.8664     55.503      6.628      0.000     259.083     476.650
treatment_period               670.4030    244.231      2.745      0.006     191.719    1149.087
treatment_1                    452.9135    238.082      1.902      0.057     -13.719     919.546
treatment_period:treatment_1  -233.9496    211.579     -1.106      0.269    -648.636     180.737
==============================================================================
Omnibus:                      257.301   Durbin-Watson:                   1.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12087.196
Skew:                           4.991   Prob(JB):                         0.00
Kurtosis:                      38.350   Cond. No.                         6.71
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [49]:
initial_roads_data = initial[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=initial_roads_data).fit(cov_type="HC3")
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     4.272
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           4.26e-05
Time:                        18:47:15   Log-Likelihood:                -1829.8
No. Observations:                 215   AIC:                             3680.
Df Residuals:                     205   BIC:                             3713.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           520.4792    232.378      2.240      0.025      65.027     975.931
C(region)[T.Мурманская область]     455.2709    397.375      1.146      0.252    -323.570    1234.112
C(region)[T.Новгородская область]  -476.2242    302.738     -1.573      0.116   -1069.580     117.131
C(region)[T.Приморский край]        -38.5566    327.908     -0.118      0.906    -681.245     604.132
C(region)[T.Республика Алтай]      -670.9337    380.590     -1.763      0.078   -1416.877      75.010
C(region)[T.Тульская область]      -197.1947    334.048     -0.590      0.555    -851.916     457.527
C(region)[T.Хабаровский край]      -234.7901    327.915     -0.716      0.474    -877.492     407.912
treatment_period                    711.6146    221.289      3.216      0.001     277.896    1145.333
treatment_1                         358.3619    204.025      1.756      0.079     -41.520     758.244
treatment_period:treatment_1       -243.8231    340.988     -0.715      0.475    -912.146     424.500
==============================================================================
Omnibus:                      259.937   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12656.417
Skew:                           5.063   Prob(JB):                         0.00
Kurtosis:                      39.198   Cond. No.                         8.48
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [50]:
initial_housing_data = initial[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=initial_housing_data).fit(cov_type="cluster", cov_kwds={'groups': initial_housing_data['region']})
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.186
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.191
Time:                        18:47:17   Log-Likelihood:                -2382.5
No. Observations:                 224   AIC:                             4773.
Df Residuals:                     220   BIC:                             4787.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3364.8444   2246.542      1.498      0.134   -1038.296    7767.985
treatment_period               811.8417   1569.663      0.517      0.605   -2264.642    3888.325
treatment_1                  -2417.5706   2127.156     -1.137      0.256   -6586.721    1751.579
treatment_period:treatment_1  -475.6193   1592.036     -0.299      0.765   -3595.952    2644.713
==============================================================================
Omnibus:                      283.839   Durbin-Watson:                   1.068
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12688.201
Skew:                           5.640   Prob(JB):                         0.00
Kurtosis:                      38.103   Cond. No.                         6.56
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [51]:
initial_housing_data = initial[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=initial_housing_data).fit(cov_type="HC3")
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     7.343
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           2.59e-09
Time:                        18:47:18   Log-Likelihood:                -2357.3
No. Observations:                 224   AIC:                             4735.
Df Residuals:                     214   BIC:                             4769.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1453.0331   2099.683      0.692      0.489   -2662.270    5568.336
C(region)[T.Мурманская область]     700.0848   1443.034      0.485      0.628   -2128.210    3528.379
C(region)[T.Новгородская область] -1545.8517   1493.002     -1.035      0.300   -4472.082    1380.379
C(region)[T.Приморский край]       -590.9555   1432.583     -0.413      0.680   -3398.766    2216.855
C(region)[T.Республика Алтай]      -720.0339   1358.548     -0.530      0.596   -3382.738    1942.670
C(region)[T.Тульская область]      -456.4485   1304.480     -0.350      0.726   -3013.182    2100.285
C(region)[T.Хабаровский край]      1.185e+04   3926.502      3.019      0.003    4157.637    1.95e+04
treatment_period                    736.6194   1790.545      0.411      0.681   -2772.784    4246.023
treatment_1                        -912.7007   1225.636     -0.745      0.456   -3314.903    1489.501
treatment_period:treatment_1         22.5607   1855.873      0.012      0.990   -3614.883    3660.005
==============================================================================
Omnibus:                      260.708   Durbin-Watson:                   1.348
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10186.476
Skew:                           4.912   Prob(JB):                         0.00
Kurtosis:                      34.542   Cond. No.                         9.05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [52]:
initial_education_data = initial[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=initial_education_data).fit(cov_type="cluster", cov_kwds={'groups': initial_education_data['region']})
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     3.006
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.117
Time:                        18:47:20   Log-Likelihood:                -2430.0
No. Observations:                 231   AIC:                             4868.
Df Residuals:                     227   BIC:                             4882.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1.353e+04   3093.990      4.373      0.000    7467.385    1.96e+04
treatment_period              1501.8115   1130.558      1.328      0.184    -714.042    3717.665
treatment_1                  -3272.9588   2608.639     -1.255      0.210   -8385.797    1839.880
treatment_period:treatment_1   376.4078   1018.688      0.370      0.712   -1620.185    2373.000
==============================================================================
Omnibus:                      121.661   Durbin-Watson:                   0.715
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              658.138
Skew:                           2.082   Prob(JB):                    1.22e-143
Kurtosis:                      10.144   Cond. No.                         6.49
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [53]:
initial_education_data = initial[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=initial_education_data).fit(cov_type="HC3")
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.429
Method:                 Least Squares   F-statistic:                     35.71
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.76e-38
Time:                        18:47:22   Log-Likelihood:                -2364.1
No. Observations:                 231   AIC:                             4748.
Df Residuals:                     221   BIC:                             4783.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          8579.9172    814.874     10.529      0.000    6982.793    1.02e+04
C(region)[T.Мурманская область]    1.347e+04    932.454     14.443      0.000    1.16e+04    1.53e+04
C(region)[T.Новгородская область] -1610.5548    709.724     -2.269      0.023   -3001.589    -219.521
C(region)[T.Приморский край]       3344.1651    837.571      3.993      0.000    1702.555    4985.775
C(region)[T.Республика Алтай]      9596.9713   1663.833      5.768      0.000    6335.918    1.29e+04
C(region)[T.Тульская область]      1632.3123    939.378      1.738      0.082    -208.835    3473.460
C(region)[T.Хабаровский край]      1.401e+04   2512.954      5.573      0.000    9080.061    1.89e+04
treatment_period                   1161.7184   1193.916      0.973      0.331   -1178.315    3501.751
treatment_1                       -2650.7367   1113.517     -2.381      0.017   -4833.190    -468.284
treatment_period:treatment_1       1194.2021   1753.839      0.681      0.496   -2243.260    4631.664
==============================================================================
Omnibus:                      119.223   Durbin-Watson:                   1.239
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1128.070
Skew:                           1.787   Prob(JB):                    1.10e-245
Kurtosis:                      13.219   Cond. No.                         8.53
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

### Models for difference

In [54]:
difference_roads_data = difference[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=difference_roads_data).fit(cov_type="cluster", cov_kwds={'groups': difference_roads_data['region']})
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                     1.894
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.232
Time:                        18:47:49   Log-Likelihood:                -1751.2
No. Observations:                 206   AIC:                             3510.
Df Residuals:                     202   BIC:                             3524.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      556.0756    214.056      2.598      0.009     136.533     975.618
treatment_period              -116.8114    153.865     -0.759      0.448    -418.381     184.759
treatment_1                   -291.3960    204.661     -1.424      0.155    -692.524     109.732
treatment_period:treatment_1   511.0047    239.131      2.137      0.033      42.317     979.693
==============================================================================
Omnibus:                       68.939   Durbin-Watson:                   1.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2415.287
Skew:                          -0.414   Prob(JB):                         0.00
Kurtosis:                      19.754   Cond. No.                         6.59
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [55]:
difference_roads_data = difference[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=difference_roads_data).fit(cov_type="HC3")
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     2.789
Date:                Tue, 25 Apr 2023   Prob (F-statistic):            0.00425
Time:                        18:47:53   Log-Likelihood:                -1742.6
No. Observations:                 206   AIC:                             3505.
Df Residuals:                     196   BIC:                             3538.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           946.6036    278.040      3.405      0.001     401.654    1491.553
C(region)[T.Мурманская область]    -142.4357    373.033     -0.382      0.703    -873.568     588.697
C(region)[T.Новгородская область]  -638.7900    249.790     -2.557      0.011   -1128.370    -149.210
C(region)[T.Приморский край]       -781.6627    292.477     -2.673      0.008   -1354.907    -208.419
C(region)[T.Республика Алтай]       170.2552    289.286      0.589      0.556    -396.734     737.244
C(region)[T.Тульская область]       -56.6808    390.059     -0.145      0.884    -821.182     707.820
C(region)[T.Хабаровский край]      -116.6728    312.123     -0.374      0.709    -728.422     495.077
treatment_period                   -241.2229    234.330     -1.029      0.303    -700.501     218.055
treatment_1                        -275.6282    204.271     -1.349      0.177    -675.991     124.735
treatment_period:treatment_1        567.0451    311.375      1.821      0.069     -43.238    1177.328
==============================================================================
Omnibus:                       67.887   Durbin-Watson:                   2.075
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2052.797
Skew:                          -0.461   Prob(JB):                         0.00
Kurtosis:                      18.437   Cond. No.                         8.91
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [56]:
difference_housing_data = difference[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=difference_housing_data).fit(cov_type="cluster", cov_kwds={'groups': difference_housing_data['region']})
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     3.848
Date:                Tue, 25 Apr 2023   Prob (F-statistic):             0.0754
Time:                        18:47:56   Log-Likelihood:                -2144.0
No. Observations:                 213   AIC:                             4296.
Df Residuals:                     209   BIC:                             4309.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      692.7611    768.507      0.901      0.367    -813.484    2199.007
treatment_period              2121.2171    932.067      2.276      0.023     294.399    3948.036
treatment_1                    319.1030    790.640      0.404      0.687   -1230.522    1868.728
treatment_period:treatment_1 -2167.7486   1118.866     -1.937      0.053   -4360.686      25.189
==============================================================================
Omnibus:                       87.364   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11954.209
Skew:                           0.048   Prob(JB):                         0.00
Kurtosis:                      39.701   Cond. No.                         6.49
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [57]:
difference_housing_data = difference[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=difference_housing_data).fit(cov_type="HC3")
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     2.049
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.108
Time:                        18:47:58   Log-Likelihood:                -2144.0
No. Observations:                 213   AIC:                             4296.
Df Residuals:                     209   BIC:                             4309.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      692.7611    805.723      0.860      0.390    -886.427    2271.949
treatment_period              2121.2171   1091.935      1.943      0.052     -18.937    4261.371
treatment_1                    319.1030    884.623      0.361      0.718   -1414.727    2052.933
treatment_period:treatment_1 -2167.7486   1166.008     -1.859      0.063   -4453.082     117.585
==============================================================================
Omnibus:                       87.364   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11954.209
Skew:                           0.048   Prob(JB):                         0.00
Kurtosis:                      39.701   Cond. No.                         6.49
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [58]:
difference_education_data = difference[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=difference_education_data).fit(cov_type="cluster", cov_kwds={'groups': difference_education_data['region']})
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7825
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.546
Time:                        18:48:00   Log-Likelihood:                -2150.0
No. Observations:                 219   AIC:                             4308.
Df Residuals:                     215   BIC:                             4321.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3117.8243   1536.194      2.030      0.042     106.940    6128.708
treatment_period              -363.6222   1635.999     -0.222      0.824   -3570.122    2842.877
treatment_1                  -1122.8627    867.170     -1.295      0.195   -2822.484     576.759
treatment_period:treatment_1    84.8193    893.781      0.095      0.924   -1666.959    1836.597
==============================================================================
Omnibus:                       98.251   Durbin-Watson:                   1.534
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              380.228
Skew:                           1.839   Prob(JB):                     2.72e-83
Kurtosis:                       8.304   Cond. No.                         6.45
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [59]:
difference_education_data = difference[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=difference_education_data).fit(cov_type="HC3")
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     5.116
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           2.90e-06
Time:                        18:48:02   Log-Likelihood:                -2130.1
No. Observations:                 219   AIC:                             4280.
Df Residuals:                     209   BIC:                             4314.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          6048.9925   1031.961      5.862      0.000    4026.385    8071.600
C(region)[T.Мурманская область]   -4271.8221    990.733     -4.312      0.000   -6213.623   -2330.021
C(region)[T.Новгородская область] -3770.6589   1074.375     -3.510      0.000   -5876.395   -1664.923
C(region)[T.Приморский край]      -5233.8532   1005.905     -5.203      0.000   -7205.391   -3262.315
C(region)[T.Республика Алтай]      -823.8995   1334.345     -0.617      0.537   -3439.167    1791.368
C(region)[T.Тульская область]     -3666.9583   1134.424     -3.232      0.001   -5890.388   -1443.528
C(region)[T.Хабаровский край]     -2047.9350   1427.152     -1.435      0.151   -4845.102     749.232
treatment_period                   -592.2510    711.294     -0.833      0.405   -1986.361     801.859
treatment_1                        -488.9449    785.915     -0.622      0.534   -2029.310    1051.420
treatment_period:treatment_1        300.0472   1166.446      0.257      0.797   -1986.146    2586.240
==============================================================================
Omnibus:                       97.685   Durbin-Watson:                   1.809
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              515.437
Skew:                           1.682   Prob(JB):                    1.19e-112
Kurtosis:                       9.721   Cond. No.                         8.98
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Lagged models

# t+1

In [60]:
with open("D:/DZ/Course_6/Diploma/Data/full_ind_data.csv", "rb") as inp:
    full_ind_data = pickle.load(inp)

In [61]:
full_ind_data.shape

(1386, 14)

In [62]:
full_ind_data.head()

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed
0,Владивостокский,Приморский край,1,1,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211
1,Владивостокский,Приморский край,1,1,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417
2,Владивостокский,Приморский край,1,1,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201
3,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074
4,Владивостокский,Приморский край,1,1,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201


In [63]:
two_time_points = pd.DataFrame()
for i in mapping.keys():
    two_time_points = two_time_points.append(full_ind_data.loc[(full_ind_data.region == i) & ((full_ind_data.year == mapping[i]) | (full_ind_data.year == mapping[i] + 6))])

two_time_points.reset_index(inplace=True, drop=True)

two_time_points["treatment_period"] = [1 if i > 2018 else 0 for i in two_time_points.year]

In [64]:
treat_1_roads = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_roads"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ treatment_period*treatment_1", data=treat_1_roads).fit(cov_type="cluster", cov_kwds={'groups': treat_1_roads['region']})
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.7108
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.580
Time:                        18:48:52   Log-Likelihood:                -852.26
No. Observations:                 182   AIC:                             1713.
Df Residuals:                     178   BIC:                             1725.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       58.0953      6.504      8.932      0.000      45.348      70.843
treatment_period                -3.8462      3.282     -1.172      0.241     -10.280       2.587
treatment_1                     -2.8301      7.756     -0.365      0.715     -18.031      12.371
treatment_period:treatment_1     3.9238      6.192      0.634      0.526      -8.212      16.059
==============================================================================
Omnibus:                       18.968   Durbin-Watson:                   1.073
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                8.544
Skew:                          -0.314   Prob(JB):                       0.0140
Kurtosis:                       2.144   Cond. No.                         6.30
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [70]:
treat_1_roads = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_roads"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ treatment_period*treatment_1 + C(region)", data=treat_1_roads).fit(cov_type="HC3")
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     10.35
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.05e-12
Time:                        18:50:06   Log-Likelihood:                -813.49
No. Observations:                 182   AIC:                             1647.
Df Residuals:                     172   BIC:                             1679.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            58.4572      4.344     13.456      0.000      49.942      66.972
C(region)[T.Мурманская область]     -24.5955      9.613     -2.559      0.011     -43.436      -5.755
C(region)[T.Новгородская область]    -2.5479      6.450     -0.395      0.693     -15.191      10.095
C(region)[T.Республика Алтай]        21.0210      6.194      3.394      0.001       8.881      33.161
C(region)[T.Республика Марий Эл]     19.4985      4.662      4.183      0.000      10.362      28.635
C(region)[T.Тульская область]        -4.8641      4.886     -0.996      0.319     -14.440       4.712
C(region)[T.Хабаровский край]       -24.3771      6.310     -3.863      0.000     -36.744     -12.010
treatment_period                     -4.6057      4.098     -1.124      0.261     -12.637       3.425
treatment_1                          -3.6117      6.190     -0.583      0.560     -15.744       8.521
treatment_period:treatment_1          4.4902      7.532      0.596      0.551     -10.272      19.252
==============================================================================
Omnibus:                        1.499   Durbin-Watson:                   1.630
Prob(Omnibus):                  0.473   Jarque-Bera (JB):                1.137
Skew:                          -0.165   Prob(JB):                        0.566
Kurtosis:                       3.202   Cond. No.                         7.78
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [65]:
treat_1_schools = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_schools"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ treatment_period*treatment_1", data=treat_1_schools).fit(cov_type="cluster", cov_kwds={'groups': treat_1_schools['region']})
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     35.64
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           0.000322
Time:                        18:48:55   Log-Likelihood:                -433.16
No. Observations:                 114   AIC:                             874.3
Df Residuals:                     110   BIC:                             885.3
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        3.8329      1.663      2.304      0.021       0.573       7.093
treatment_period                11.9893      1.663      7.208      0.000       8.729      15.249
treatment_1                     -1.5722      1.991     -0.789      0.430      -5.475       2.331
treatment_period:treatment_1    -8.0751      3.148     -2.565      0.010     -14.245      -1.905
==============================================================================
Omnibus:                       78.710   Durbin-Watson:                   2.099
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              346.580
Skew:                           2.539   Prob(JB):                     5.51e-76
Kurtosis:                       9.868   Cond. No.                         7.68
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [71]:
treat_1_schools = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_schools"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ treatment_period*treatment_1 + C(region)", data=treat_1_schools).fit(cov_type="HC3")
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     2.406
Date:                Tue, 25 Apr 2023   Prob (F-statistic):             0.0161
Time:                        18:50:35   Log-Likelihood:                -420.38
No. Observations:                 114   AIC:                             860.8
Df Residuals:                     104   BIC:                             888.1
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            28.8500     29.911      0.965      0.335     -29.774      87.474
C(region)[T.Новгородская область]   -26.6535     29.933     -0.890      0.373     -85.321      32.014
C(region)[T.Приморский край]        -28.6918     29.914     -0.959      0.337     -87.322      29.938
C(region)[T.Республика Алтай]       -25.6457     30.128     -0.851      0.395     -84.695      33.404
C(region)[T.Республика Марий Эл]    -25.2971     29.987     -0.844      0.399     -84.071      33.477
C(region)[T.Тульская область]       -23.5393     30.008     -0.784      0.433     -82.353      35.275
C(region)[T.Хабаровский край]        -7.1833     34.597     -0.208      0.836     -74.993      60.626
treatment_period                     12.6179      8.636      1.461      0.144      -4.308      29.544
treatment_1                          -0.4746      1.327     -0.358      0.721      -3.076       2.126
treatment_period:treatment_1         -9.3469      8.744     -1.069      0.285     -26.486       7.792
==============================================================================
Omnibus:                       53.529   Durbin-Watson:                   2.365
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.245
Skew:                           1.780   Prob(JB):                     1.44e-33
Kurtosis:                       7.377   Cond. No.                         23.2
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [66]:
treat_1_sewerage = two_time_points[["raion", "region", "treatment_1", "treatment_period", "sewage_grid_fixed"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ treatment_period*treatment_1", data=treat_1_sewerage).fit(cov_type="cluster", cov_kwds={'groups': treat_1_sewerage['region']})
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.036
Method:                 Least Squares   F-statistic:                     35.72
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           0.000841
Time:                        18:48:56   Log-Likelihood:                 111.93
No. Observations:                  63   AIC:                            -215.9
Df Residuals:                      59   BIC:                            -207.3
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0217      0.007      3.010      0.003       0.008       0.036
treatment_period                -0.0068      0.009     -0.790      0.429      -0.024       0.010
treatment_1                     -0.0049      0.006     -0.754      0.451      -0.018       0.008
treatment_period:treatment_1    -0.0063      0.008     -0.797      0.426      -0.022       0.009
==============================================================================
Omnibus:                       88.719   Durbin-Watson:                   2.219
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1129.587
Skew:                           4.263   Prob(JB):                    5.17e-246
Kurtosis:                      21.911   Cond. No.                         6.01
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [72]:
treat_1_sewerage = two_time_points[["raion", "region", "treatment_1", "treatment_period", "sewage_grid_fixed"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ treatment_period*treatment_1 + C(region)", data=treat_1_sewerage).fit(cov_type="HC3")
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                     1.599
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.147
Time:                        18:50:57   Log-Likelihood:                 114.98
No. Observations:                  63   AIC:                            -212.0
Df Residuals:                      54   BIC:                            -192.7
Df Model:                           8                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0098      0.010      0.987      0.324      -0.010       0.029
C(region)[T.Мурманская область]      -0.0022      0.005     -0.420      0.674      -0.012       0.008
C(region)[T.Новгородская область]     0.0122      0.010      1.241      0.214      -0.007       0.031
C(region)[T.Приморский край]          0.0357      0.027      1.329      0.184      -0.017       0.088
C(region)[T.Тульская область]        -0.0025      0.010     -0.258      0.797      -0.022       0.017
C(region)[T.Хабаровский край]         0.0028      0.006      0.429      0.668      -0.010       0.015
treatment_period                      0.0028      0.008      0.334      0.739      -0.014       0.019
treatment_1                          -0.0067      0.017     -0.393      0.694      -0.040       0.027
treatment_period:treatment_1         -0.0032      0.016     -0.200      0.842      -0.035       0.029
==============================================================================
Omnibus:                       73.515   Durbin-Watson:                   2.421
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              634.865
Skew:                           3.383   Prob(JB):                    1.38e-138
Kurtosis:                      17.002   Cond. No.                         11.4
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [67]:
treat_1_heat = two_time_points[["raion", "region", "treatment_1", "treatment_period", "heat_grid_fixed"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ treatment_period*treatment_1", data=treat_1_heat).fit(cov_type="HC3") # т.к. в выборке только один регион
heat.summary()

D:\Anaconda\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
D:\Anaconda\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.034
Method:                 Least Squares   F-statistic:                   0.04536
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.833
Time:                        18:48:59   Log-Likelihood:                 79.514
No. Observations:                  30   AIC:                            -155.0
Df Residuals:                      28   BIC:                            -152.2
Df Model:                           1                                         
Covariance Type:                  HC3                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0240      0.004      5.779      0.000       0.016       0.032
treatment_period                      0          0        nan        nan           0           0
treatment_1                      0.0015      0.007      0.213      0.831      -0.012       0.015
treatment_period:treatment_1          0          0        nan        nan           0           0
==============================================================================
Omnibus:                       25.382   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.336
Skew:                           1.949   Prob(JB):                     2.36e-10
Kurtosis:                       7.503   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [68]:
treat_1_water = two_time_points[["raion", "region", "treatment_1", "treatment_period", "water_grid_fixed"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ treatment_period*treatment_1", data=treat_1_water).fit(cov_type="cluster", cov_kwds={'groups': treat_1_water['region']})
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                     12.87
Date:                Tue, 25 Apr 2023   Prob (F-statistic):            0.00869
Time:                        18:49:00   Log-Likelihood:                 149.47
No. Observations:                 123   AIC:                            -290.9
Df Residuals:                     119   BIC:                            -279.7
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0319      0.015      2.172      0.030       0.003       0.061
treatment_period                 0.0069      0.016      0.434      0.664      -0.024       0.038
treatment_1                     -0.0196      0.015     -1.312      0.189      -0.049       0.010
treatment_period:treatment_1    -0.0097      0.016     -0.590      0.555      -0.042       0.022
==============================================================================
Omnibus:                      203.978   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16759.290
Skew:                           6.907   Prob(JB):                         0.00
Kurtosis:                      58.491   Cond. No.                         6.10
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [73]:
treat_1_water = two_time_points[["raion", "region", "treatment_1", "treatment_period", "water_grid_fixed"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ treatment_period*treatment_1 + C(region)", data=treat_1_water).fit(cov_type="HC3")
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     3.132
Date:                Tue, 25 Apr 2023   Prob (F-statistic):            0.00308
Time:                        18:53:20   Log-Likelihood:                 155.28
No. Observations:                 123   AIC:                            -292.6
Df Residuals:                     114   BIC:                            -267.2
Df Model:                           8                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0229      0.009      2.596      0.009       0.006       0.040
C(region)[T.Мурманская область]       0.0719      0.065      1.109      0.267      -0.055       0.199
C(region)[T.Новгородская область]    -0.0138      0.008     -1.729      0.084      -0.029       0.002
C(region)[T.Приморский край]          0.0040      0.009      0.450      0.653      -0.013       0.021
C(region)[T.Республика Алтай]        -0.0097      0.019     -0.519      0.604      -0.046       0.027
C(region)[T.Хабаровский край]         0.0138      0.015      0.922      0.356      -0.016       0.043
treatment_period                      0.0115      0.019      0.612      0.541      -0.025       0.048
treatment_1                          -0.0231      0.016     -1.426      0.154      -0.055       0.009
treatment_period:treatment_1         -0.0041      0.024     -0.169      0.866      -0.051       0.043
==============================================================================
Omnibus:                      187.283   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11334.060
Skew:                           6.032   Prob(JB):                         0.00
Kurtosis:                      48.453   Cond. No.                         6.79
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Fin t+1

In [74]:
with open("D:/DZ/Course_6/Diploma/Data/financial_data.csv", "rb") as inp:
    temp_fin = pickle.load(inp)

In [75]:
with open("D:/DZ/Course_5/Курсовая/data/data_for_models.csv", "rb") as inp:
    data_for_models = pickle.load(inp)

In [76]:
mapping = {"Приморский край": 2016, "Мурманская область": 2016, "Костромская область": 2015, "Республика Алтай": 2014, "Республика Марий Эл": 2014, "Хабаровский край": 2014, 
           "Тульская область": 2014, "Новгородская область": 2016}

In [77]:
actual, initial, difference = temp_fin.loc[temp_fin.budget_type == "actual_spending"], temp_fin.loc[temp_fin.budget_type == "initial_budget"], temp_fin.loc[temp_fin.budget_type == "разница"]

In [78]:
actual = pd.pivot(actual, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
initial = pd.pivot(initial, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
difference = pd.pivot(difference, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()

In [79]:
for i in actual, initial, difference:
    i.rename(columns={"дорожное хозяйство (дорожные фонды)": "roads", "жилищно-коммунальное хозяйство": "housing", "образование": "education"}, inplace=True)

In [80]:
actual.head()

sub_indicator,region,raion,year,roads,housing,education
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN


In [81]:
frames = [actual, initial, difference]

In [82]:
for i in range(len(frames)):
    temp_fin_data = pd.DataFrame()
    for j in mapping.keys():
        temp_fin_data = temp_fin_data.append(frames[i].loc[(frames[i].region == j) & ((frames[i].year == mapping[j]) | (frames[i].year == mapping[j] + 6))])
    temp_fin_data["treatment_period"] = [1 if i > 2018 else 0 for i in temp_fin_data.year]
    frames[i] = pd.merge(temp_fin_data, data_for_models, on=["region", "raion"], how="inner")

In [83]:
#frames[0]

In [84]:
actual, initial, difference = frames

### Models for actual budgets

In [85]:
actual_roads_data = actual[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=actual_roads_data).fit(cov_type="cluster", cov_kwds={'groups': actual_roads_data['region']})
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     3.246
Date:                Tue, 25 Apr 2023   Prob (F-statistic):             0.0903
Time:                        18:56:25   Log-Likelihood:                -1760.4
No. Observations:                 203   AIC:                             3529.
Df Residuals:                     199   BIC:                             3542.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1002.3017    204.024      4.913      0.000     602.422    1402.181
treatment_period               585.7705    307.909      1.902      0.057     -17.720    1189.261
treatment_1                    208.8738    234.638      0.890      0.373    -251.009     668.756
treatment_period:treatment_1    84.5872    205.724      0.411      0.681    -318.624     487.798
==============================================================================
Omnibus:                       98.606   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              337.558
Skew:                           2.068   Prob(JB):                     5.02e-74
Kurtosis:                       7.776   Cond. No.                         6.21
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [87]:
actual_roads_data = actual[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=actual_roads_data).fit(cov_type="HC3")
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     4.436
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.26e-05
Time:                        18:56:47   Log-Likelihood:                -1746.2
No. Observations:                 203   AIC:                             3514.
Df Residuals:                     192   BIC:                             3551.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1202.5801    281.207      4.276      0.000     651.425    1753.735
C(region)[T.Мурманская область]    1172.4153    926.704      1.265      0.206    -643.891    2988.722
C(region)[T.Новгородская область]  -805.0537    294.533     -2.733      0.006   -1382.327    -227.780
C(region)[T.Приморский край]       -678.6603    285.172     -2.380      0.017   -1237.586    -119.734
C(region)[T.Республика Алтай]       625.4679    471.529      1.326      0.185    -298.713    1549.648
C(region)[T.Республика Марий Эл]   -415.8056    310.981     -1.337      0.181   -1025.318     193.706
C(region)[T.Тульская область]      -502.8096    328.300     -1.532      0.126   -1146.265     140.646
C(region)[T.Хабаровский край]       354.1925    394.239      0.898      0.369    -418.503    1126.888
treatment_period                    254.2888    314.508      0.809      0.419    -362.136     870.713
treatment_1                         249.3544    258.431      0.965      0.335    -257.160     755.869
treatment_period:treatment_1        150.6349    567.092      0.266      0.791    -960.846    1262.116
==============================================================================
Omnibus:                       85.039   Durbin-Watson:                   2.108
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              273.830
Skew:                           1.761   Prob(JB):                     3.46e-60
Kurtosis:                       7.469   Cond. No.                         8.61
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [46]:
actual_housing_data = actual[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=actual_housing_data).fit(cov_type="cluster", cov_kwds={'groups': actual_housing_data['region']})
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     1.268
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.357
Time:                        06:19:49   Log-Likelihood:                -2159.8
No. Observations:                 202   AIC:                             4328.
Df Residuals:                     198   BIC:                             4341.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3624.2584   2255.306      1.607      0.108    -796.059    8044.576
treatment_period              5384.0650   2877.668      1.871      0.061    -256.060     1.1e+04
treatment_1                  -1706.8698   2109.744     -0.809      0.418   -5841.892    2428.152
treatment_period:treatment_1 -5448.8490   3071.499     -1.774      0.076   -1.15e+04     571.178
==============================================================================
Omnibus:                      269.496   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16078.804
Skew:                           5.818   Prob(JB):                         0.00
Kurtosis:                      45.130   Cond. No.                         6.21
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [89]:
actual_housing_data = actual[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=actual_housing_data).fit(cov_type="HC3")
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     5.594
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           2.60e-07
Time:                        18:57:48   Log-Likelihood:                -2134.7
No. Observations:                 202   AIC:                             4291.
Df Residuals:                     191   BIC:                             4328.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          2050.3165   1000.528      2.049      0.040      89.317    4011.316
C(region)[T.Мурманская область]    1395.5159   1147.043      1.217      0.224    -852.646    3643.678
C(region)[T.Новгородская область] -1417.8691   1009.888     -1.404      0.160   -3397.213     561.475
C(region)[T.Приморский край]      -1106.6367    903.516     -1.225      0.221   -2877.496     664.222
C(region)[T.Республика Алтай]      -147.0979    983.335     -0.150      0.881   -2074.399    1780.203
C(region)[T.Республика Марий Эл]   -934.9331    772.708     -1.210      0.226   -2449.413     579.547
C(region)[T.Тульская область]      1540.6333   1238.008      1.244      0.213    -885.817    3967.084
C(region)[T.Хабаровский край]      1.259e+04   3391.178      3.714      0.000    5947.232    1.92e+04
treatment_period                   2778.4877   2613.274      1.063      0.288   -2343.435    7900.410
treatment_1                        -695.2176    769.409     -0.904      0.366   -2203.231     812.796
treatment_period:treatment_1      -4043.7913   2903.742     -1.393      0.164   -9735.022    1647.439
==============================================================================
Omnibus:                      265.478   Durbin-Watson:                   1.370
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17978.353
Skew:                           5.582   Prob(JB):                         0.00
Kurtosis:                      47.849   Cond. No.                         8.90
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [48]:
actual_education_data = actual[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=actual_education_data).fit(cov_type="cluster", cov_kwds={'groups': actual_education_data['region']})
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     1.538
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.287
Time:                        06:20:09   Log-Likelihood:                -2157.7
No. Observations:                 204   AIC:                             4323.
Df Residuals:                     200   BIC:                             4337.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1.567e+04   2940.651      5.327      0.000    9902.493    2.14e+04
treatment_period              4553.0537   3490.794      1.304      0.192   -2288.776    1.14e+04
treatment_1                  -3653.9746   2606.139     -1.402      0.161   -8761.912    1453.963
treatment_period:treatment_1 -4900.7094   2489.901     -1.968      0.049   -9780.826     -20.593
==============================================================================
Omnibus:                      133.408   Durbin-Watson:                   0.816
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              947.120
Skew:                           2.536   Prob(JB):                    2.17e-206
Kurtosis:                      12.257   Cond. No.                         6.22
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [90]:
actual_education_data = actual[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=actual_education_data).fit(cov_type="HC3")
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     8.080
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           4.16e-05
Time:                        18:58:04   Log-Likelihood:                -2157.7
No. Observations:                 204   AIC:                             4323.
Df Residuals:                     200   BIC:                             4337.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1.567e+04    955.678     16.393      0.000    1.38e+04    1.75e+04
treatment_period              4553.0537   2048.077      2.223      0.026     538.896    8567.211
treatment_1                  -3653.9746   1210.673     -3.018      0.003   -6026.850   -1281.100
treatment_period:treatment_1 -4900.7094   2630.956     -1.863      0.063   -1.01e+04     255.870
==============================================================================
Omnibus:                      133.408   Durbin-Watson:                   0.816
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              947.120
Skew:                           2.536   Prob(JB):                    2.17e-206
Kurtosis:                      12.257   Cond. No.                         6.22
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

### Models for initial budgets

In [50]:
initial_roads_data = initial[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=initial_roads_data).fit(cov_type="cluster", cov_kwds={'groups': initial_roads_data['region']})
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     139.8
Date:                Fri, 31 Mar 2023   Prob (F-statistic):           6.11e-06
Time:                        06:20:17   Log-Likelihood:                -1357.7
No. Observations:                 166   AIC:                             2723.
Df Residuals:                     162   BIC:                             2736.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      367.8664     55.620      6.614      0.000     258.852     476.881
treatment_period               710.2978    151.810      4.679      0.000     412.755    1007.840
treatment_1                    452.9135    238.586      1.898      0.058     -14.707     920.534
treatment_period:treatment_1    41.1128    247.912      0.166      0.868    -444.785     527.011
==============================================================================
Omnibus:                      102.231   Durbin-Watson:                   1.750
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              608.817
Skew:                           2.295   Prob(JB):                    6.27e-133
Kurtosis:                      11.183   Cond. No.                         6.05
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [91]:
initial_roads_data = initial[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=initial_roads_data).fit(cov_type="HC3")
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     5.031
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           5.87e-06
Time:                        18:58:39   Log-Likelihood:                -1350.6
No. Observations:                 166   AIC:                             2721.
Df Residuals:                     156   BIC:                             2752.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           292.4572    129.290      2.262      0.024      39.054     545.860
C(region)[T.Мурманская область]     852.8230    524.116      1.627      0.104    -174.425    1880.071
C(region)[T.Новгородская область]  -116.4613    145.112     -0.803      0.422    -400.875     167.953
C(region)[T.Приморский край]         57.7520    147.095      0.393      0.695    -230.549     346.053
C(region)[T.Республика Алтай]      -615.6991    233.195     -2.640      0.008   -1072.753    -158.645
C(region)[T.Тульская область]       -21.7223    280.960     -0.077      0.938    -572.393     528.949
C(region)[T.Хабаровский край]       146.3106    230.714      0.634      0.526    -305.881     598.502
treatment_period                    849.3940    211.028      4.025      0.000     435.787    1263.000
treatment_1                         412.3973    193.615      2.130      0.033      32.919     791.876
treatment_period:treatment_1         82.1087    400.892      0.205      0.838    -703.626     867.843
==============================================================================
Omnibus:                       90.974   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              485.085
Skew:                           2.022   Prob(JB):                    4.63e-106
Kurtosis:                      10.334   Cond. No.                         7.71
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [52]:
initial_housing_data = initial[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=initial_housing_data).fit(cov_type="cluster", cov_kwds={'groups': initial_housing_data['region']})
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.025
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.446
Time:                        06:20:24   Log-Likelihood:                -1870.5
No. Observations:                 174   AIC:                             3749.
Df Residuals:                     170   BIC:                             3762.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3364.8444   2250.982      1.495      0.135   -1046.999    7776.688
treatment_period              3769.0129   3643.895      1.034      0.301   -3372.891    1.09e+04
treatment_1                  -2417.5706   2131.361     -1.134      0.257   -6594.961    1759.819
treatment_period:treatment_1 -3646.7376   3595.794     -1.014      0.311   -1.07e+04    3400.890
==============================================================================
Omnibus:                      206.254   Durbin-Watson:                   1.168
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5728.977
Skew:                           4.866   Prob(JB):                         0.00
Kurtosis:                      29.372   Cond. No.                         5.99
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [93]:
initial_housing_data = initial[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=initial_housing_data).fit(cov_type="HC3")
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     5.025
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           5.51e-06
Time:                        18:59:13   Log-Likelihood:                -1854.3
No. Observations:                 174   AIC:                             3729.
Df Residuals:                     164   BIC:                             3760.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1570.5993   2855.671      0.550      0.582   -4026.412    7167.611
C(region)[T.Мурманская область]     469.5922   2605.261      0.180      0.857   -4636.626    5575.811
C(region)[T.Новгородская область] -1365.9905   2721.019     -0.502      0.616   -6699.090    3967.109
C(region)[T.Приморский край]       -695.6624   2512.627     -0.277      0.782   -5620.321    4228.996
C(region)[T.Республика Алтай]      -943.6730   1494.775     -0.631      0.528   -3873.377    1986.031
C(region)[T.Тульская область]        80.4521   1484.836      0.054      0.957   -2829.772    2990.676
C(region)[T.Хабаровский край]      1.122e+04   3930.546      2.855      0.004    3517.289    1.89e+04
treatment_period                   1908.3191   3248.802      0.587      0.557   -4459.216    8275.854
treatment_1                       -1048.6792   1247.740     -0.840      0.401   -3494.205    1396.847
treatment_period:treatment_1      -2602.9168   2666.185     -0.976      0.329   -7828.543    2622.709
==============================================================================
Omnibus:                      193.190   Durbin-Watson:                   1.404
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4932.389
Skew:                           4.393   Prob(JB):                         0.00
Kurtosis:                      27.559   Cond. No.                         8.46
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [54]:
initial_education_data = initial[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=initial_education_data).fit(cov_type="cluster", cov_kwds={'groups': initial_education_data['region']})
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                    0.7065
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.582
Time:                        06:20:29   Log-Likelihood:                -1909.3
No. Observations:                 181   AIC:                             3827.
Df Residuals:                     177   BIC:                             3839.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1.353e+04   3099.685      4.365      0.000    7456.224    1.96e+04
treatment_period              2498.9782   2890.661      0.865      0.387   -3166.613    8164.569
treatment_1                  -3272.9588   2613.440     -1.252      0.210   -8395.208    1849.290
treatment_period:treatment_1 -2350.6876   2387.668     -0.985      0.325   -7030.430    2329.055
==============================================================================
Omnibus:                      112.927   Durbin-Watson:                   0.812
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              707.826
Skew:                           2.367   Prob(JB):                    1.98e-154
Kurtosis:                      11.453   Cond. No.                         5.99
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [95]:
initial_education_data = initial[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=initial_education_data).fit(cov_type="HC3")
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     25.24
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           3.23e-27
Time:                        18:59:42   Log-Likelihood:                -1866.3
No. Observations:                 181   AIC:                             3753.
Df Residuals:                     171   BIC:                             3785.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          8821.7577   1186.444      7.435      0.000    6496.370    1.11e+04
C(region)[T.Мурманская область]    1.174e+04   1329.630      8.830      0.000    9134.674    1.43e+04
C(region)[T.Новгородская область] -1350.3387   1281.685     -1.054      0.292   -3862.395    1161.717
C(region)[T.Приморский край]       2660.0850   1220.674      2.179      0.029     267.608    5052.562
C(region)[T.Республика Алтай]      8901.3685   1431.715      6.217      0.000    6095.259    1.17e+04
C(region)[T.Тульская область]      2650.1420   1052.485      2.518      0.012     587.309    4712.975
C(region)[T.Хабаровский край]      1.402e+04   2504.382      5.599      0.000    9113.989    1.89e+04
treatment_period                    520.8062   2001.872      0.260      0.795   -3402.791    4444.403
treatment_1                       -2641.8828   1121.991     -2.355      0.019   -4840.945    -442.820
treatment_period:treatment_1        -82.0136   2294.471     -0.036      0.971   -4579.095    4415.068
==============================================================================
Omnibus:                      100.421   Durbin-Watson:                   1.225
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              825.921
Skew:                           1.902   Prob(JB):                    4.50e-180
Kurtosis:                      12.749   Cond. No.                         7.81
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

### Models for difference

In [56]:
difference_roads_data = difference[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=difference_roads_data).fit(cov_type="cluster", cov_kwds={'groups': difference_roads_data['region']})
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                     4.312
Date:                Fri, 31 Mar 2023   Prob (F-statistic):             0.0607
Time:                        06:20:34   Log-Likelihood:                -1175.0
No. Observations:                 140   AIC:                             2358.
Df Residuals:                     136   BIC:                             2370.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      556.0756    214.815      2.589      0.010     135.046     977.105
treatment_period                29.7667    212.218      0.140      0.888    -386.174     445.707
treatment_1                   -291.3960    205.386     -1.419      0.156    -693.946     111.154
treatment_period:treatment_1   652.4823    227.826      2.864      0.004     205.951    1099.014
==============================================================================
Omnibus:                       92.850   Durbin-Watson:                   2.049
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              490.085
Skew:                           2.473   Prob(JB):                    3.80e-107
Kurtosis:                      10.717   Cond. No.                         6.25
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [100]:
difference_roads_data = difference[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=difference_roads_data).fit(cov_type="HC3")
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     2.284
Date:                Tue, 25 Apr 2023   Prob (F-statistic):             0.0206
Time:                        19:03:06   Log-Likelihood:                -1168.3
No. Observations:                 140   AIC:                             2357.
Df Residuals:                     130   BIC:                             2386.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           910.0785    333.373      2.730      0.006     256.680    1563.477
C(region)[T.Мурманская область]     205.6187    819.805      0.251      0.802   -1401.169    1812.407
C(region)[T.Новгородская область]  -742.7973    337.715     -2.199      0.028   -1404.706     -80.888
C(region)[T.Приморский край]       -712.7427    313.420     -2.274      0.023   -1327.035     -98.450
C(region)[T.Республика Алтай]      -228.2966    292.051     -0.782      0.434    -800.705     344.112
C(region)[T.Тульская область]      -332.0470    396.578     -0.837      0.402   -1109.326     445.232
C(region)[T.Хабаровский край]        34.5707    324.202      0.107      0.915    -600.853     669.995
treatment_period                   -287.8047    325.665     -0.884      0.377    -926.096     350.487
treatment_1                        -273.6311    219.758     -1.245      0.213    -704.348     157.086
treatment_period:treatment_1        646.7174    547.932      1.180      0.238    -427.210    1720.645
==============================================================================
Omnibus:                       80.714   Durbin-Watson:                   2.177
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              354.205
Skew:                           2.154   Prob(JB):                     1.22e-77
Kurtosis:                       9.494   Cond. No.                         8.17
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [58]:
difference_housing_data = difference[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=difference_housing_data).fit(cov_type="cluster", cov_kwds={'groups': difference_housing_data['region']})
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                     266.7
Date:                Fri, 31 Mar 2023   Prob (F-statistic):           9.00e-07
Time:                        06:20:39   Log-Likelihood:                -1461.3
No. Observations:                 146   AIC:                             2931.
Df Residuals:                     142   BIC:                             2942.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      692.7611    771.068      0.898      0.369    -818.505    2204.027
treatment_period               869.0813    810.023      1.073      0.283    -718.534    2456.697
treatment_1                    319.1030    793.275      0.402      0.687   -1235.687    1873.893
treatment_period:treatment_1 -1268.6071    804.216     -1.577      0.115   -2844.841     307.627
==============================================================================
Omnibus:                      175.586   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10724.903
Skew:                          -4.214   Prob(JB):                         0.00
Kurtosis:                      44.133   Cond. No.                         6.27
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [99]:
difference_housing_data = difference[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=difference_housing_data).fit(cov_type="HC3")
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                 -0.031
Method:                 Least Squares   F-statistic:                     1.648
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.108
Time:                        19:02:54   Log-Likelihood:                -1459.1
No. Observations:                 146   AIC:                             2938.
Df Residuals:                     136   BIC:                             2968.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          -632.2387   3007.827     -0.210      0.834   -6527.471    5262.993
C(region)[T.Мурманская область]    1358.0787   2780.301      0.488      0.625   -4091.211    6807.368
C(region)[T.Новгородская область]  1047.9130   2925.951      0.358      0.720   -4686.846    6782.672
C(region)[T.Приморский край]        692.6799   2743.102      0.253      0.801   -4683.702    6069.062
C(region)[T.Республика Алтай]      2138.1149   2062.515      1.037      0.300   -1904.341    6180.570
C(region)[T.Тульская область]      2248.2210   2723.149      0.826      0.409   -3089.054    7585.496
C(region)[T.Хабаровский край]      2534.7268   2432.655      1.042      0.297   -2233.190    7302.643
treatment_period                    413.1272   1882.155      0.219      0.826   -3275.828    4102.083
treatment_1                         442.6659    817.363      0.542      0.588   -1159.336    2044.668
treatment_period:treatment_1       -779.4265   1073.206     -0.726      0.468   -2882.871    1324.018
==============================================================================
Omnibus:                      174.562   Durbin-Watson:                   1.827
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9692.041
Skew:                          -4.219   Prob(JB):                         0.00
Kurtosis:                      42.013   Cond. No.                         8.94
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [60]:
difference_education_data = difference[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=difference_education_data).fit(cov_type="cluster", cov_kwds={'groups': difference_education_data['region']})
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     2.506
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.156
Time:                        06:20:48   Log-Likelihood:                -1486.3
No. Observations:                 151   AIC:                             2981.
Df Residuals:                     147   BIC:                             2993.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3117.8243   1541.075      2.023      0.043      97.372    6138.277
treatment_period              1744.1283   1773.408      0.983      0.325   -1731.688    5219.945
treatment_1                  -1122.8627    869.926     -1.291      0.197   -2827.885     582.160
treatment_period:treatment_1  -517.2890   1640.224     -0.315      0.752   -3732.070    2697.491
==============================================================================
Omnibus:                       80.322   Durbin-Watson:                   1.486
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.647
Skew:                           2.102   Prob(JB):                     4.67e-64
Kurtosis:                       8.356   Cond. No.                         6.32
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [98]:
difference_education_data = difference[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=difference_education_data).fit(cov_type="HC3")
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     2.319
Date:                Tue, 25 Apr 2023   Prob (F-statistic):             0.0779
Time:                        19:00:41   Log-Likelihood:                -1486.3
No. Observations:                 151   AIC:                             2981.
Df Residuals:                     147   BIC:                             2993.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3117.8243    552.244      5.646      0.000    2035.447    4200.202
treatment_period              1744.1283    980.177      1.779      0.075    -176.983    3665.240
treatment_1                  -1122.8627    915.614     -1.226      0.220   -2917.433     671.707
treatment_period:treatment_1  -517.2890   1862.760     -0.278      0.781   -4168.231    3133.653
==============================================================================
Omnibus:                       80.322   Durbin-Watson:                   1.486
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.647
Skew:                           2.102   Prob(JB):                     4.67e-64
Kurtosis:                       8.356   Cond. No.                         6.32
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Goods t-1 

In [101]:
with open("D:/DZ/Course_6/Diploma/Data/full_ind_data.csv", "rb") as inp:
    full_ind_data = pickle.load(inp)

In [102]:
full_ind_data.shape

(1386, 14)

In [103]:
full_ind_data.head()

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed
0,Владивостокский,Приморский край,1,1,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211
1,Владивостокский,Приморский край,1,1,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417
2,Владивостокский,Приморский край,1,1,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201
3,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074
4,Владивостокский,Приморский край,1,1,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201


In [104]:
two_time_points = pd.DataFrame()
for i in mapping.keys():
    two_time_points = two_time_points.append(full_ind_data.loc[(full_ind_data.region == i) & ((full_ind_data.year == mapping[i]) | (full_ind_data.year == mapping[i] + 4))])

two_time_points.reset_index(inplace=True, drop=True)

two_time_points["treatment_period"] = [1 if i > 2018 else 0 for i in two_time_points.year]

In [105]:
treat_1_roads = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_roads"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ treatment_period*treatment_1", data=treat_1_roads).fit(cov_type="cluster", cov_kwds={'groups': treat_1_roads['region']})
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                    0.6194
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.628
Time:                        19:03:41   Log-Likelihood:                -992.80
No. Observations:                 212   AIC:                             1994.
Df Residuals:                     208   BIC:                             2007.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       56.8639      8.547      6.653      0.000      40.113      73.615
treatment_period                -3.8808      8.570     -0.453      0.651     -20.678      12.917
treatment_1                     -0.7700      8.875     -0.087      0.931     -18.165      16.625
treatment_period:treatment_1    -9.0089     10.436     -0.863      0.388     -29.462      11.445
==============================================================================
Omnibus:                       27.217   Durbin-Watson:                   0.954
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                9.794
Skew:                          -0.261   Prob(JB):                      0.00747
Kurtosis:                       2.086   Cond. No.                         6.30
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [106]:
treat_1_roads = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_roads"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ treatment_period*treatment_1 + C(region)", data=treat_1_roads).fit(cov_type="HC3")
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     12.31
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.78e-15
Time:                        19:04:02   Log-Likelihood:                -948.54
No. Observations:                 212   AIC:                             1917.
Df Residuals:                     202   BIC:                             1951.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            58.4242      4.559     12.816      0.000      49.489      67.359
C(region)[T.Мурманская область]     -19.4847      6.233     -3.126      0.002     -31.701      -7.268
C(region)[T.Новгородская область]    -4.5657      4.942     -0.924      0.356     -14.251       5.120
C(region)[T.Республика Алтай]        22.0507      6.686      3.298      0.001       8.947      35.154
C(region)[T.Республика Марий Эл]     16.5262      5.527      2.990      0.003       5.693      27.360
C(region)[T.Тульская область]        -4.7637      5.547     -0.859      0.390     -15.635       6.108
C(region)[T.Хабаровский край]       -31.2983      6.863     -4.561      0.000     -44.749     -17.848
treatment_period                      0.2270      5.073      0.045      0.964      -9.715      10.169
treatment_1                          -4.0869      4.715     -0.867      0.386     -13.329       5.155
treatment_period:treatment_1         -5.3475      7.417     -0.721      0.471     -19.885       9.190
==============================================================================
Omnibus:                        2.366   Durbin-Watson:                   1.433
Prob(Omnibus):                  0.306   Jarque-Bera (JB):                2.023
Skew:                          -0.161   Prob(JB):                        0.364
Kurtosis:                       3.353   Cond. No.                         7.94
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [68]:
treat_1_schools = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_schools"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ treatment_period*treatment_1", data=treat_1_schools).fit(cov_type="cluster", cov_kwds={'groups': treat_1_schools['region']})
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     16.26
Date:                Fri, 31 Mar 2023   Prob (F-statistic):            0.00155
Time:                        06:27:05   Log-Likelihood:                -506.87
No. Observations:                 139   AIC:                             1022.
Df Residuals:                     135   BIC:                             1033.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        3.3126      1.239      2.675      0.007       0.885       5.740
treatment_period                16.6874      2.949      5.659      0.000      10.908      22.467
treatment_1                     -0.4715      1.483     -0.318      0.751      -3.379       2.436
treatment_period:treatment_1    -4.3285      3.226     -1.342      0.180     -10.652       1.995
==============================================================================
Omnibus:                      115.653   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              868.057
Skew:                           3.142   Prob(JB):                    3.19e-189
Kurtosis:                      13.506   Cond. No.                         9.34
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [108]:
treat_1_schools = two_time_points[["raion", "region", "treatment_1", "treatment_period", "bad_schools"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ treatment_period*treatment_1 + C(region)", data=treat_1_schools).fit(cov_type="HC3")
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     3.407
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           0.000559
Time:                        19:05:23   Log-Likelihood:                -485.74
No. Observations:                 139   AIC:                             993.5
Df Residuals:                     128   BIC:                             1026.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            28.8500     29.911      0.965      0.335     -29.774      87.474
C(region)[T.Мурманская область]     -41.3588     30.958     -1.336      0.182    -102.035      19.318
C(region)[T.Новгородская область]   -26.1607     29.938     -0.874      0.382     -84.837      32.516
C(region)[T.Приморский край]        -29.5569     29.920     -0.988      0.323     -88.199      29.085
C(region)[T.Республика Алтай]       -24.0569     30.014     -0.802      0.423     -82.883      34.769
C(region)[T.Республика Марий Эл]    -27.4589     29.935     -0.917      0.359     -86.130      31.212
C(region)[T.Тульская область]       -24.1932     30.012     -0.806      0.420     -83.016      34.630
C(region)[T.Хабаровский край]        -7.1833     34.597     -0.208      0.836     -74.993      60.626
treatment_period                     19.3484      5.533      3.497      0.000       8.504      30.193
treatment_1                           0.6879      1.556      0.442      0.658      -2.361       3.737
treatment_period:treatment_1         -3.1275      8.169     -0.383      0.702     -19.138      12.883
==============================================================================
Omnibus:                       74.543   Durbin-Watson:                   2.189
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              282.993
Skew:                           2.041   Prob(JB):                     3.54e-62
Kurtosis:                       8.675   Cond. No.                         27.9
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [70]:
treat_1_sewerage = two_time_points[["raion", "region", "treatment_1", "treatment_period", "sewage_grid_fixed"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ treatment_period*treatment_1", data=treat_1_sewerage).fit(cov_type="cluster", cov_kwds={'groups': treat_1_sewerage['region']})
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                     2.061
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.224
Time:                        06:27:13   Log-Likelihood:                 121.47
No. Observations:                 107   AIC:                            -234.9
Df Residuals:                     103   BIC:                            -224.3
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0167      0.006      2.873      0.004       0.005       0.028
treatment_period                 0.0208      0.022      0.942      0.346      -0.022       0.064
treatment_1                     -0.0018      0.005     -0.381      0.703      -0.011       0.008
treatment_period:treatment_1    -0.0202      0.025     -0.821      0.412      -0.068       0.028
==============================================================================
Omnibus:                      194.974   Durbin-Watson:                   1.512
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19272.373
Skew:                           7.570   Prob(JB):                         0.00
Kurtosis:                      66.981   Cond. No.                         5.84
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [110]:
treat_1_sewerage = two_time_points[["raion", "region", "treatment_1", "treatment_period", "sewage_grid_fixed"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ treatment_period*treatment_1 + C(region)", data=treat_1_sewerage).fit(cov_type="HC3")
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.8551
Date:                Tue, 25 Apr 2023   Prob (F-statistic):              0.557
Time:                        19:06:14   Log-Likelihood:                 124.29
No. Observations:                 107   AIC:                            -230.6
Df Residuals:                      98   BIC:                            -206.5
Df Model:                           8                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0120      0.009      1.307      0.191      -0.006       0.030
C(region)[T.Мурманская область]      -0.0044      0.013     -0.336      0.737      -0.030       0.021
C(region)[T.Новгородская область]     0.0012      0.015      0.078      0.938      -0.028       0.030
C(region)[T.Приморский край]          0.0404      0.022      1.806      0.071      -0.003       0.084
C(region)[T.Тульская область]        -0.0012      0.006     -0.213      0.831      -0.013       0.010
C(region)[T.Хабаровский край]        -0.0012      0.008     -0.162      0.871      -0.016       0.014
treatment_period                      0.0066      0.031      0.212      0.832      -0.054       0.067
treatment_1                          -0.0056      0.011     -0.499      0.618      -0.027       0.016
treatment_period:treatment_1         -0.0165      0.033     -0.507      0.612      -0.080       0.047
==============================================================================
Omnibus:                      191.293   Durbin-Watson:                   1.611
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18325.347
Skew:                           7.302   Prob(JB):                         0.00
Kurtosis:                      65.427   Cond. No.                         16.4
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [72]:
treat_1_heat = two_time_points[["raion", "region", "treatment_1", "treatment_period", "heat_grid_fixed"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ treatment_period*treatment_1", data=treat_1_heat).fit(cov_type="HC3") # т.к. в выборке только один регион
heat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.7255
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.541
Time:                        06:27:18   Log-Likelihood:                 121.44
No. Observations:                  58   AIC:                            -234.9
Df Residuals:                      54   BIC:                            -226.6
Df Model:                           3                                         
Covariance Type:                  HC3                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0240      0.004      5.779      0.000       0.016       0.032
treatment_period                 0.0117      0.013      0.915      0.360      -0.013       0.037
treatment_1                      0.0015      0.007      0.213      0.831      -0.012       0.015
treatment_period:treatment_1    -0.0172      0.014     -1.202      0.230      -0.045       0.011
==============================================================================
Omnibus:                       97.363   Durbin-Watson:                   2.103
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2219.782
Skew:                           4.920   Prob(JB):                         0.00
Kurtosis:                      31.666   Cond. No.                         6.25
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [74]:
treat_1_water = two_time_points[["raion", "region", "treatment_1", "treatment_period", "water_grid_fixed"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ treatment_period*treatment_1", data=treat_1_water).fit(cov_type="cluster", cov_kwds={'groups': treat_1_water['region']})
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.9209
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.495
Time:                        06:27:24   Log-Likelihood:                 227.54
No. Observations:                 158   AIC:                            -447.1
Df Residuals:                     154   BIC:                            -434.8
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0303      0.012      2.586      0.010       0.007       0.053
treatment_period                -0.0105      0.008     -1.273      0.203      -0.027       0.006
treatment_1                     -0.0175      0.012     -1.472      0.141      -0.041       0.006
treatment_period:treatment_1     0.0142      0.009      1.506      0.132      -0.004       0.033
==============================================================================
Omnibus:                      300.886   Durbin-Watson:                   1.672
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            74115.309
Skew:                           9.543   Prob(JB):                         0.00
Kurtosis:                     107.373   Cond. No.                         5.92
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [111]:
treat_1_water = two_time_points[["raion", "region", "treatment_1", "treatment_period", "water_grid_fixed"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ treatment_period*treatment_1 + C(region)", data=treat_1_water).fit(cov_type="HC3")
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     3.692
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           0.000584
Time:                        19:07:03   Log-Likelihood:                 234.59
No. Observations:                 158   AIC:                            -451.2
Df Residuals:                     149   BIC:                            -423.6
Df Model:                           8                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0165      0.005      3.430      0.001       0.007       0.026
C(region)[T.Мурманская область]       0.0529      0.038      1.388      0.165      -0.022       0.128
C(region)[T.Новгородская область]  7.791e-05      0.008      0.010      0.992      -0.015       0.015
C(region)[T.Приморский край]          0.0140      0.008      1.697      0.090      -0.002       0.030
C(region)[T.Республика Алтай]         0.0024      0.006      0.393      0.694      -0.009       0.014
C(region)[T.Хабаровский край]         0.0198      0.007      2.904      0.004       0.006       0.033
treatment_period                     -0.0138      0.017     -0.816      0.414      -0.047       0.019
treatment_1                          -0.0193      0.012     -1.594      0.111      -0.043       0.004
treatment_period:treatment_1          0.0142      0.014      0.980      0.327      -0.014       0.042
==============================================================================
Omnibus:                      291.276   Durbin-Watson:                   1.822
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            64295.645
Skew:                           8.970   Prob(JB):                         0.00
Kurtosis:                     100.183   Cond. No.                         9.59
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Fin t-1

In [112]:
with open("D:/DZ/Course_6/Diploma/Data/financial_data.csv", "rb") as inp:
    temp_fin = pickle.load(inp)

In [113]:
with open("D:/DZ/Course_5/Курсовая/data/data_for_models.csv", "rb") as inp:
    data_for_models = pickle.load(inp)

In [114]:
mapping = {"Приморский край": 2016, "Мурманская область": 2016, "Костромская область": 2015, "Республика Алтай": 2014, "Республика Марий Эл": 2014, "Хабаровский край": 2014, 
           "Тульская область": 2014, "Новгородская область": 2016}

In [115]:
actual, initial, difference = temp_fin.loc[temp_fin.budget_type == "actual_spending"], temp_fin.loc[temp_fin.budget_type == "initial_budget"], temp_fin.loc[temp_fin.budget_type == "разница"]

In [116]:
actual = pd.pivot(actual, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
initial = pd.pivot(initial, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
difference = pd.pivot(difference, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()

In [117]:
for i in actual, initial, difference:
    i.rename(columns={"дорожное хозяйство (дорожные фонды)": "roads", "жилищно-коммунальное хозяйство": "housing", "образование": "education"}, inplace=True)

In [118]:
actual.head()

sub_indicator,region,raion,year,roads,housing,education
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN


In [119]:
frames = [actual, initial, difference]

In [120]:
for i in range(len(frames)):
    temp_fin_data = pd.DataFrame()
    for j in mapping.keys():
        temp_fin_data = temp_fin_data.append(frames[i].loc[(frames[i].region == j) & ((frames[i].year == mapping[j]) | (frames[i].year == mapping[j] + 4))])
    temp_fin_data["treatment_period"] = [1 if i > 2018 else 0 for i in temp_fin_data.year]
    frames[i] = pd.merge(temp_fin_data, data_for_models, on=["region", "raion"], how="inner")

In [121]:
#frames[0]

In [122]:
actual, initial, difference = frames

### Models for actual budgets

In [100]:
actual_roads_data = actual[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=actual_roads_data).fit(cov_type="cluster", cov_kwds={'groups': actual_roads_data['region']})
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     17.48
Date:                Fri, 31 Mar 2023   Prob (F-statistic):            0.00124
Time:                        06:33:02   Log-Likelihood:                -2310.9
No. Observations:                 268   AIC:                             4630.
Df Residuals:                     264   BIC:                             4644.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1037.2126    142.809      7.263      0.000     757.313    1317.112
treatment_period               205.9884    183.470      1.123      0.262    -153.606     565.583
treatment_1                    273.0508    214.326      1.274      0.203    -147.020     693.122
treatment_period:treatment_1   745.6776    278.473      2.678      0.007     199.880    1291.476
==============================================================================
Omnibus:                      125.826   Durbin-Watson:                   1.829
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              456.715
Skew:                           2.063   Prob(JB):                    6.69e-100
Kurtosis:                       7.887   Cond. No.                         6.24
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [123]:
actual_roads_data = actual[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=actual_roads_data).fit(cov_type="HC3")
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     6.928
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.34e-09
Time:                        19:08:17   Log-Likelihood:                -2293.0
No. Observations:                 268   AIC:                             4608.
Df Residuals:                     257   BIC:                             4647.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1097.8440    286.777      3.828      0.000     535.771    1659.917
C(region)[T.Мурманская область]     858.1579    609.103      1.409      0.159    -335.661    2051.977
C(region)[T.Новгородская область]  -818.9471    314.977     -2.600      0.009   -1436.290    -201.604
C(region)[T.Приморский край]       -495.4664    282.140     -1.756      0.079   -1048.450      57.517
C(region)[T.Республика Алтай]       659.1368    452.600      1.456      0.145    -227.942    1546.216
C(region)[T.Республика Марий Эл]   -385.8781    313.988     -1.229      0.219   -1001.284     229.527
C(region)[T.Тульская область]       349.2724    409.642      0.853      0.394    -453.610    1152.155
C(region)[T.Хабаровский край]       200.6692    366.024      0.548      0.584    -516.725     918.063
treatment_period                    613.0406    234.882      2.610      0.009     152.680    1073.401
treatment_1                         198.1882    221.786      0.894      0.372    -236.505     632.881
treatment_period:treatment_1        708.8060    473.480      1.497      0.134    -219.197    1636.809
==============================================================================
Omnibus:                      128.281   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              519.864
Skew:                           2.047   Prob(JB):                    1.30e-113
Kurtosis:                       8.458   Cond. No.                         10.2
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [102]:
actual_housing_data = actual[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=actual_housing_data).fit(cov_type="cluster", cov_kwds={'groups': actual_housing_data['region']})
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.130
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.400
Time:                        06:33:08   Log-Likelihood:                -2820.5
No. Observations:                 265   AIC:                             5649.
Df Residuals:                     261   BIC:                             5663.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     5195.9009   3369.984      1.542      0.123   -1409.147    1.18e+04
treatment_period             -3533.8044   3376.752     -1.047      0.295   -1.02e+04    3084.508
treatment_1                  -3309.5010   3288.117     -1.007      0.314   -9754.093    3135.091
treatment_period:treatment_1  3809.6278   3277.569      1.162      0.245   -2614.289    1.02e+04
==============================================================================
Omnibus:                      360.710   Durbin-Watson:                   1.134
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26698.134
Skew:                           6.436   Prob(JB):                         0.00
Kurtosis:                      50.458   Cond. No.                         6.23
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [125]:
actual_housing_data = actual[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=actual_housing_data).fit(cov_type="HC3")
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     8.109
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           2.32e-11
Time:                        19:08:44   Log-Likelihood:                -2783.7
No. Observations:                 265   AIC:                             5589.
Df Residuals:                     254   BIC:                             5629.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          2681.5789    548.371      4.890      0.000    1606.792    3756.365
C(region)[T.Мурманская область]    1123.9193    679.458      1.654      0.098    -207.794    2455.632
C(region)[T.Новгородская область] -2358.2869    536.791     -4.393      0.000   -3410.378   -1306.196
C(region)[T.Приморский край]      -1383.2145    495.506     -2.792      0.005   -2354.389    -412.040
C(region)[T.Республика Алтай]        65.9807    661.530      0.100      0.921   -1230.594    1362.555
C(region)[T.Республика Марий Эл]  -1094.4199    532.620     -2.055      0.040   -2138.335     -50.504
C(region)[T.Тульская область]       407.5894    701.451      0.581      0.561    -967.230    1782.408
C(region)[T.Хабаровский край]      1.385e+04   3810.325      3.636      0.000    6384.562    2.13e+04
treatment_period                    429.6206    353.078      1.217      0.224    -262.400    1121.642
treatment_1                       -1464.9488    836.397     -1.751      0.080   -3104.258     174.360
treatment_period:treatment_1       1682.1706    915.011      1.838      0.066    -111.218    3475.560
==============================================================================
Omnibus:                      340.242   Durbin-Watson:                   1.438
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24146.610
Skew:                           5.785   Prob(JB):                         0.00
Kurtosis:                      48.310   Cond. No.                         10.6
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [104]:
actual_education_data = actual[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=actual_education_data).fit(cov_type="cluster", cov_kwds={'groups': actual_education_data['region']})
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     2.224
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.173
Time:                        06:33:12   Log-Likelihood:                -2807.3
No. Observations:                 268   AIC:                             5623.
Df Residuals:                     264   BIC:                             5637.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1.655e+04   3363.027      4.920      0.000    9954.053    2.31e+04
treatment_period             -2310.0688   3426.639     -0.674      0.500   -9026.158    4406.020
treatment_1                  -4960.2119   3065.058     -1.618      0.106    -1.1e+04    1047.191
treatment_period:treatment_1  4717.0493   3059.507      1.542      0.123   -1279.475    1.07e+04
==============================================================================
Omnibus:                      167.629   Durbin-Watson:                   0.849
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1308.855
Skew:                           2.491   Prob(JB):                    6.11e-285
Kurtosis:                      12.612   Cond. No.                         6.24
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [126]:
actual_education_data = actual[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=actual_education_data).fit(cov_type="HC3")
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     48.58
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.34e-53
Time:                        19:09:15   Log-Likelihood:                -2733.1
No. Observations:                 268   AIC:                             5488.
Df Residuals:                     257   BIC:                             5528.
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.718e+04   1119.658     15.343      0.000     1.5e+04    1.94e+04
C(region)[T.Мурманская область]    3260.5656   1252.252      2.604      0.009     806.196    5714.935
C(region)[T.Новгородская область] -7697.7382   1280.330     -6.012      0.000   -1.02e+04   -5188.337
C(region)[T.Приморский край]      -4631.7451   1215.955     -3.809      0.000   -7014.972   -2248.518
C(region)[T.Республика Алтай]      3988.0321   1580.378      2.523      0.012     890.548    7085.516
C(region)[T.Республика Марий Эл]  -7418.2619   1163.429     -6.376      0.000   -9698.541   -5137.983
C(region)[T.Тульская область]     -4608.3854   1275.958     -3.612      0.000   -7109.216   -2107.555
C(region)[T.Хабаровский край]      9231.6082   2600.496      3.550      0.000    4134.730    1.43e+04
treatment_period                   1869.1523    916.721      2.039      0.041      72.412    3665.893
treatment_1                       -3042.6530    890.818     -3.416      0.001   -4788.624   -1296.682
treatment_period:treatment_1       1917.0603   1721.667      1.113      0.265   -1457.346    5291.466
==============================================================================
Omnibus:                      172.610   Durbin-Watson:                   1.252
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1850.105
Skew:                           2.441   Prob(JB):                         0.00
Kurtosis:                      14.910   Cond. No.                         10.2
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

### Models for initial budgets

In [106]:
initial_roads_data = initial[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=initial_roads_data).fit(cov_type="cluster", cov_kwds={'groups': initial_roads_data['region']})
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     5.807
Date:                Fri, 31 Mar 2023   Prob (F-statistic):             0.0330
Time:                        06:33:17   Log-Likelihood:                -2097.7
No. Observations:                 225   AIC:                             4203.
Df Residuals:                     221   BIC:                             4217.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      430.3440     69.020      6.235      0.000     295.068     565.620
treatment_period              1035.2302    647.607      1.599      0.110    -234.057    2304.517
treatment_1                    461.7647    151.345      3.051      0.002     165.134     758.395
treatment_period:treatment_1  -559.4565    549.861     -1.017      0.309   -1637.165     518.252
==============================================================================
Omnibus:                      467.765   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           300066.199
Skew:                          12.728   Prob(JB):                         0.00
Kurtosis:                     180.085   Cond. No.                         5.95
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [127]:
initial_roads_data = initial[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=initial_roads_data).fit(cov_type="HC3")
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     4.241
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           4.49e-05
Time:                        19:09:54   Log-Likelihood:                -2094.9
No. Observations:                 225   AIC:                             4210.
Df Residuals:                     215   BIC:                             4244.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1013.0107    681.062      1.487      0.137    -321.847    2347.868
C(region)[T.Мурманская область]    -191.4671    945.697     -0.202      0.840   -2044.998    1662.064
C(region)[T.Новгородская область] -1216.9999    951.136     -1.280      0.201   -3081.192     647.192
C(region)[T.Приморский край]       -822.2233    866.758     -0.949      0.343   -2521.038     876.591
C(region)[T.Республика Алтай]      -720.5663    687.091     -1.049      0.294   -2067.240     626.107
C(region)[T.Тульская область]      -344.3629    706.176     -0.488      0.626   -1728.443    1039.717
C(region)[T.Хабаровский край]      -444.0029    696.171     -0.638      0.524   -1808.473     920.468
treatment_period                   1082.7448    627.485      1.726      0.084    -147.103    2312.593
treatment_1                         391.0129    175.781      2.224      0.026      46.489     735.537
treatment_period:treatment_1       -569.3600    716.609     -0.795      0.427   -1973.889     835.169
==============================================================================
Omnibus:                      464.685   Durbin-Watson:                   2.089
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           289617.806
Skew:                          12.556   Prob(JB):                         0.00
Kurtosis:                     176.960   Cond. No.                         8.71
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [108]:
initial_housing_data = initial[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=initial_housing_data).fit(cov_type="cluster", cov_kwds={'groups': initial_housing_data['region']})
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.185
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.191
Time:                        06:33:21   Log-Likelihood:                -2553.2
No. Observations:                 232   AIC:                             5114.
Df Residuals:                     228   BIC:                             5128.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     5143.0897   3456.233      1.488      0.137   -1631.002    1.19e+04
treatment_period              -751.1320   3736.532     -0.201      0.841   -8074.600    6572.336
treatment_1                  -4146.5428   3362.287     -1.233      0.217   -1.07e+04    2443.418
treatment_period:treatment_1  1042.4301   3747.428      0.278      0.781   -6302.393    8387.254
==============================================================================
Omnibus:                      359.120   Durbin-Watson:                   1.637
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49604.434
Skew:                           7.538   Prob(JB):                         0.00
Kurtosis:                      73.030   Cond. No.                         5.92
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [129]:
initial_housing_data = initial[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=initial_housing_data).fit(cov_type="HC3")
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     3.312
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           0.000816
Time:                        19:10:54   Log-Likelihood:                -2542.0
No. Observations:                 232   AIC:                             5104.
Df Residuals:                     222   BIC:                             5138.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          5875.3750   4273.079      1.375      0.169   -2499.706    1.43e+04
C(region)[T.Мурманская область]   -3860.3156   4786.699     -0.806      0.420   -1.32e+04    5521.441
C(region)[T.Новгородская область] -6378.0185   4998.599     -1.276      0.202   -1.62e+04    3419.056
C(region)[T.Приморский край]      -4381.4749   4712.485     -0.930      0.352   -1.36e+04    4854.826
C(region)[T.Республика Алтай]     -4693.4327   4216.370     -1.113      0.266    -1.3e+04    3570.501
C(region)[T.Тульская область]     -3813.8270   4123.114     -0.925      0.355   -1.19e+04    4267.327
C(region)[T.Хабаровский край]      7695.5494   5448.605      1.412      0.158   -2983.520    1.84e+04
treatment_period                   2292.4152   2644.401      0.867      0.386   -2890.516    7475.346
treatment_1                       -2166.1103   1074.462     -2.016      0.044   -4272.018     -60.203
treatment_period:treatment_1      -1305.5711   3050.052     -0.428      0.669   -7283.564    4672.422
==============================================================================
Omnibus:                      362.294   Durbin-Watson:                   1.805
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            56616.618
Skew:                           7.597   Prob(JB):                         0.00
Kurtosis:                      78.007   Cond. No.                         9.42
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [110]:
initial_education_data = initial[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=initial_education_data).fit(cov_type="cluster", cov_kwds={'groups': initial_education_data['region']})
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     20.15
Date:                Fri, 31 Mar 2023   Prob (F-statistic):            0.00155
Time:                        06:33:26   Log-Likelihood:                -2521.7
No. Observations:                 241   AIC:                             5051.
Df Residuals:                     237   BIC:                             5065.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1.455e+04   3201.616      4.546      0.000    8279.494    2.08e+04
treatment_period             -2316.6760   3185.335     -0.727      0.467   -8559.818    3926.466
treatment_1                  -4315.1778   2932.556     -1.471      0.141   -1.01e+04    1432.527
treatment_period:treatment_1  6000.1093   2956.163      2.030      0.042     206.136    1.18e+04
==============================================================================
Omnibus:                      135.075   Durbin-Watson:                   0.859
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              850.237
Skew:                           2.200   Prob(JB):                    2.36e-185
Kurtosis:                      11.081   Cond. No.                         5.93
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [130]:
initial_education_data = initial[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=initial_education_data).fit(cov_type="HC3")
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.399
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     37.18
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           2.97e-40
Time:                        19:11:24   Log-Likelihood:                -2464.8
No. Observations:                 241   AIC:                             4950.
Df Residuals:                     231   BIC:                             4984.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          8082.0334    635.858     12.710      0.000    6835.774    9328.293
C(region)[T.Мурманская область]    1.291e+04    825.533     15.639      0.000    1.13e+04    1.45e+04
C(region)[T.Новгородская область]  -630.9055    822.513     -0.767      0.443   -2243.001     981.191
C(region)[T.Приморский край]       4044.8675    887.843      4.556      0.000    2304.727    5785.008
C(region)[T.Республика Алтай]      7147.1836   1202.289      5.945      0.000    4790.740    9503.628
C(region)[T.Тульская область]      2989.3473   1023.966      2.919      0.004     982.412    4996.283
C(region)[T.Хабаровский край]      1.399e+04   2438.799      5.735      0.000    9207.569    1.88e+04
treatment_period                   1531.9568    730.428      2.097      0.036     100.345    2963.569
treatment_1                       -2829.3090    956.169     -2.959      0.003   -4703.365    -955.253
treatment_period:treatment_1       3216.5325   1757.945      1.830      0.067    -228.976    6662.041
==============================================================================
Omnibus:                      136.705   Durbin-Watson:                   1.225
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1336.894
Skew:                           2.036   Prob(JB):                    4.98e-291
Kurtosis:                      13.796   Cond. No.                         8.76
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

### Models for difference

In [112]:
difference_roads_data = difference[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1", data=difference_roads_data).fit(cov_type="cluster", cov_kwds={'groups': difference_roads_data['region']})
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                     3.708
Date:                Fri, 31 Mar 2023   Prob (F-statistic):             0.0807
Time:                        06:33:31   Log-Likelihood:                -1688.8
No. Observations:                 200   AIC:                             3386.
Df Residuals:                     196   BIC:                             3399.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      564.9017    157.129      3.595      0.000     256.935     872.869
treatment_period              -100.7923     81.939     -1.230      0.219    -261.390      59.806
treatment_1                   -107.9718    188.408     -0.573      0.567    -477.245     261.301
treatment_period:treatment_1   498.0429    262.578      1.897      0.058     -16.601    1012.687
==============================================================================
Omnibus:                      152.843   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1419.778
Skew:                           2.984   Prob(JB):                    5.00e-309
Kurtosis:                      14.608   Cond. No.                         5.95
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [131]:
difference_roads_data = difference[["raion", "region", "treatment_1", "treatment_period", "roads"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ treatment_period*treatment_1 + C(region)", data=difference_roads_data).fit(cov_type="HC3")
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     3.237
Date:                Tue, 25 Apr 2023   Prob (F-statistic):            0.00111
Time:                        19:11:51   Log-Likelihood:                -1681.4
No. Observations:                 200   AIC:                             3383.
Df Residuals:                     190   BIC:                             3416.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           985.6518    332.015      2.969      0.003     334.915    1636.389
C(region)[T.Мурманская область]    -381.4418    530.950     -0.718      0.473   -1422.085     659.201
C(region)[T.Новгородская область]  -809.2881    329.590     -2.455      0.014   -1455.273    -163.303
C(region)[T.Приморский край]       -748.4726    300.161     -2.494      0.013   -1336.778    -160.167
C(region)[T.Республика Алтай]      -633.1327    348.353     -1.818      0.069   -1315.892      49.627
C(region)[T.Тульская область]        39.6891    488.893      0.081      0.935    -918.524     997.903
C(region)[T.Хабаровский край]      -289.8850    371.086     -0.781      0.435   -1017.200     437.430
treatment_period                    151.7821    207.908      0.730      0.465    -255.711     559.275
treatment_1                        -185.4024    219.333     -0.845      0.398    -615.287     244.482
treatment_period:treatment_1        569.3662    373.339      1.525      0.127    -162.366    1301.098
==============================================================================
Omnibus:                      150.760   Durbin-Watson:                   2.125
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1434.999
Skew:                           2.912   Prob(JB):                         0.00
Kurtosis:                      14.760   Cond. No.                         9.73
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [114]:
difference_housing_data = difference[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1", data=difference_housing_data).fit(cov_type="cluster", cov_kwds={'groups': difference_housing_data['region']})
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                     1.444
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.320
Time:                        06:33:35   Log-Likelihood:                -2069.4
No. Observations:                 205   AIC:                             4147.
Df Residuals:                     201   BIC:                             4160.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     1335.6531    920.972      1.450      0.147    -469.420    3140.726
treatment_period             -1732.2042   1210.236     -1.431      0.152   -4104.224     639.815
treatment_1                   -367.2823    910.229     -0.404      0.687   -2151.298    1416.733
treatment_period:treatment_1  1436.2144   1235.685      1.162      0.245    -985.684    3858.113
==============================================================================
Omnibus:                      159.388   Durbin-Watson:                   2.183
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13722.585
Skew:                          -2.123   Prob(JB):                         0.00
Kurtosis:                      42.856   Cond. No.                         5.95
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [132]:
difference_housing_data = difference[["raion", "region", "treatment_1", "treatment_period", "housing"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ treatment_period*treatment_1 + C(region)", data=difference_housing_data).fit(cov_type="HC3")
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     2.398
Date:                Tue, 25 Apr 2023   Prob (F-statistic):             0.0134
Time:                        19:12:19   Log-Likelihood:                -2064.9
No. Observations:                 205   AIC:                             4150.
Df Residuals:                     195   BIC:                             4183.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          -572.2445   3383.566     -0.169      0.866   -7203.912    6059.423
C(region)[T.Мурманская область]    1047.2231   3008.510      0.348      0.728   -4849.348    6943.794
C(region)[T.Новгородская область]   854.1651   3055.264      0.280      0.780   -5134.042    6842.373
C(region)[T.Приморский край]        208.3635   3050.308      0.068      0.946   -5770.131    6186.858
C(region)[T.Республика Алтай]      2642.0578   3293.102      0.802      0.422   -3812.303    9096.419
C(region)[T.Тульская область]      1628.7658   3062.426      0.532      0.595   -4373.478    7631.010
C(region)[T.Хабаровский край]      3894.7393   3607.368      1.080      0.280   -3175.572     1.1e+04
treatment_period                   -368.8568   1202.858     -0.307      0.759   -2726.415    1988.701
treatment_1                         221.2092    731.158      0.303      0.762   -1211.835    1654.253
treatment_period:treatment_1        970.9872   1348.629      0.720      0.472   -1672.278    3614.252
==============================================================================
Omnibus:                      164.462   Durbin-Watson:                   2.272
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12248.381
Skew:                          -2.298   Prob(JB):                         0.00
Kurtosis:                      40.588   Cond. No.                         11.4
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [116]:
difference_education_data = difference[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ treatment_period*treatment_1", data=difference_education_data).fit(cov_type="cluster", cov_kwds={'groups': difference_education_data['region']})
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     2.849
Date:                Fri, 31 Mar 2023   Prob (F-statistic):              0.127
Time:                        06:33:40   Log-Likelihood:                -2085.7
No. Observations:                 212   AIC:                             4179.
Df Residuals:                     208   BIC:                             4193.
Df Model:                           3                                         
Covariance Type:              cluster                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                     3578.2144   1211.829      2.953      0.003    1203.073    5953.356
treatment_period             -1146.9399    770.827     -1.488      0.137   -2657.733     363.853
treatment_1                  -1757.0061    756.961     -2.321      0.020   -3240.622    -273.390
treatment_period:treatment_1  -445.7983    899.007     -0.496      0.620   -2207.819    1316.223
==============================================================================
Omnibus:                       61.226   Durbin-Watson:                   1.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              137.435
Skew:                           1.338   Prob(JB):                     1.43e-30
Kurtosis:                       5.898   Cond. No.                         5.99
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [133]:
difference_education_data = difference[["raion", "region", "treatment_1", "treatment_period", "education"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ treatment_period*treatment_1 + C(region)", data=difference_education_data).fit(cov_type="HC3")
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     13.19
Date:                Tue, 25 Apr 2023   Prob (F-statistic):           1.68e-16
Time:                        19:12:42   Log-Likelihood:                -2051.2
No. Observations:                 212   AIC:                             4122.
Df Residuals:                     202   BIC:                             4156.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          8753.7016    974.750      8.980      0.000    6843.227    1.07e+04
C(region)[T.Мурманская область]   -7497.7726    982.157     -7.634      0.000   -9422.766   -5572.780
C(region)[T.Новгородская область] -6714.0872   1083.007     -6.199      0.000   -8836.741   -4591.433
C(region)[T.Приморский край]      -8189.1087    992.999     -8.247      0.000   -1.01e+04   -6242.867
C(region)[T.Республика Алтай]     -3255.2804   1189.330     -2.737      0.006   -5586.324    -924.237
C(region)[T.Тульская область]     -6871.2576   1184.368     -5.802      0.000   -9192.576   -4549.939
C(region)[T.Хабаровский край]     -4693.0053   1363.292     -3.442      0.001   -7365.010   -2021.001
treatment_period                    422.4274    694.382      0.608      0.543    -938.537    1783.392
treatment_1                        -852.9290    615.218     -1.386      0.166   -2058.735     352.877
treatment_period:treatment_1       -780.4239   1288.415     -0.606      0.545   -3305.671    1744.823
==============================================================================
Omnibus:                       84.915   Durbin-Watson:                   1.551
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              433.893
Skew:                           1.477   Prob(JB):                     6.05e-95
Kurtosis:                       9.356   Cond. No.                         9.96
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""